In [1]:
import os
import bisect
import itertools
import math
import numpy as np
import pandas as pd
import requests
import pickle
import time
from datetime import timedelta, datetime as dt
import pytz
from itertools import zip_longest
from pytz import timezone
from pprint import pprint
import pandas_datareader as pdr
import pandas_datareader.data as web
from collections import namedtuple, Counter
from operator import itemgetter
from sklearn import preprocessing

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import fundamentalanalysis as fa
# import streamlit as st

import holidays
import pandas_datareader.data as web
import yahoo_fin.stock_info as si
from urllib.request import Request, urlopen
from html_table_parser.parser import HTMLTableParser 
# import investpy
# import streamlit as st

from load_data import *
from get_data import get_tickers
from info import TA_PERIODS
from functions import get_super, get_sub
import statistics

pd.options.plotting.backend = "plotly"
# pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format


ImportError: DLL load failed while importing lib: The specified procedure could not be found.

In [2]:
from talib import BBANDS, MACD, RSI
from load_data import *

In [ ]:
print(statistics.harmonic_mean([2.5, 3, 10]))
# print(statistics.harmonic_mean([2.5 / 0.5, 3 / 0.3, 10 / 0.2]))

3.6


In [1]:
def weighted_harmonic_mean(array, weights):
    if len(array) == len(weights):
        d = sum([w / x for x, w in zip(array, weights)])
        n = sum(weights)
        return n / d
    else:
        return "Weights missing from array item(s)"

In [3]:
weighted_harmonic_mean([4, 3, 10], [1, 1, 1])

4.390243902439025

In [3]:
rf_rates = get_rf_data()
SPY_df = get_SPY_data()
SPY_info_df = get_SPY_info()
ticker_info = get_ticker_info()
ticker_list = SPY_info_df.index.to_list()
sector_list = SPY_info_df['Sector'].unique()
first_date = SPY_df.iloc[0].name
last_date = SPY_df.iloc[-1].name
yr_ago = last_date - timedelta(days=365)
# TTM_ratios, ratios_data_report = load_TTM_ratios()

In [11]:
def convert_to_timestamp(x):
    """Convert date objects to integers"""
    
    return time.mktime(x.timetuple())

In [12]:
def date_format(date):
    if date.hour == 0:
        return date.strftime('%d/%m/%y')
    else:
        return date.strftime('%d/%m/%y %H:%M')

In [13]:
eastern_tz = timezone('EST')
end = dt.now().astimezone(eastern_tz).replace(tzinfo=None)
# end = dt(2023, 5, 6)
weekday = end.weekday()
hr_bool = end.hour < 16
days = 1 if hr_bool else 0
print(f'end: {end}')

if weekday >= 5:
    days += weekday - 5
elif weekday == 0 and hr_bool:
    days += 2

end -= timedelta(days)
intervals = ['1d', '1m', '5m']
dfs = []

for interval in intervals:
    start = end - timedelta(7 - days) if interval == '1m' else end - timedelta(60 - days)
    f_end = end - timedelta(1)
    # df = yf.download('A', start=start, end=f_end, interval=interval, progress=False)
    # dfs.append(df)
    # print(f'weekday: {weekday}')
    print(f'{interval} - days: {days}, timedelta: {dt.now() - start}, start: {date_format(start)}, end: {date_format(end)}, f_end: {date_format(f_end)}')
hr_bool

end: 2023-09-05 09:36:44.222332
1d - days: 1, timedelta: 60 days, 8:00:00.001995, start: 07/07/23 09:36, end: 04/09/23 09:36, f_end: 03/09/23 09:36
1m - days: 1, timedelta: 7 days, 8:00:00.002996, start: 29/08/23 09:36, end: 04/09/23 09:36, f_end: 03/09/23 09:36
5m - days: 1, timedelta: 60 days, 8:00:00.002996, start: 07/07/23 09:36, end: 04/09/23 09:36, f_end: 03/09/23 09:36


True

# Technical Analysis

#### To Do:

- Trend Analysis
- MAs - find appropriate MAs for all timeframes
- Volume Analysis
- **Need to add ability to switch to any timeframe within the bounds of the data**
- **Add an option to enable a slider that allows one to view any points along the dataset**
- Calculate Piotroski F-Scores as they could be useful in identifying value
  in securities in the accumulation stage.
- Identify seasonality in prices, which can be caused by factors such as 
  "window dressing" by institutions with upcoming earnings reports.

## Trend Analysis

In [14]:
from scipy.signal import find_peaks, peak_prominences, argrelextrema, argrelmax, argrelmin
from sklearn import preprocessing

pd.options.display.float_format = "{:,.4f}".format

### The Definition & Interaction of Trends

A trend is a period in which a price moves in an irregular but persistent direction.
The three most widely followed
trends are primary, intermediate, and short-term.

__Primary__

The primary trend generally lasts between 9 months and 2 years, and is
a reflection of investors’ attitudes toward unfolding fundamentals in the
business cycle. The business cycle extends statistically from trough to
trough for approximately 3.6 years, so it follows that rising and falling
primary trends (bull and bear markets) last for 1 to 2 years. Since building 
up takes longer than tearing down, bull markets generally last longer
than bear markets. The direction of the secular or very long-term trend
will also affect the magnitude and duration of a primary trend. Those that
move in the direction of the secular trend will generally experience greater
magnitude and duration than those that move in the opposite direction.

__Intermediate__

Anyone who has looked at prices on a chart will notice that they do not move
in a straight line. A primary upswing is interrupted by several re actions
along the way. These countercyclical trends within the confines of a primary
bull market are known as intermediate price movements. They last anywhere
from 6 weeks to as long as 9 months, sometimes even longer, but rarely
shorter. Countercyclical intermediate trends are typically very deceptive,
often being founded on very believable but false assumptions. For example, 
an intermediate rally during a bear market in equities may very well be
founded on a couple of unexpectedly positive economic numbers, which
make it appear that the economy will avoid that much-feared recession.
When subsequent numbers are reported and found to be wanting, the bear
market resumes.
It is important to have an idea of the direction and maturity of the
primary trend, but an analysis of intermediate trends is also helpful for
improving success rates in trading, as well as for determining when the
primary movement may have run its course.

__Short-Term Trends__

Short-term trends typically last 3 to 6 weeks, sometimes shorter and sometimes 
longer. They interrupt the course of the intermediate cycle, just as
the intermediate-term trend in terrupts primary price movements. Short-
term trends are shown in the market cycle model (Figure 1.1) as a dashed
line. They are usually influenced by random news events and are far more
difficult to identify than their intermediate or primary counterparts.



__Major Technical Principles__
- The direction of the primary trend will affect the 
character of intermediate and short-term trends.
- Surprises occur in the direction of the main trend, i.e., on the upside in a 
bull market and on the downside in a bear market.
- Never go for perfection; always shoot for consistency.
- The significance of a peak-and-trough
reversal is determined by the du ration and magnitude of the rallies and
reactions in question.


Maybe use linreg gradient to determine primary and secondary trend

### Trend Changepoints

In [15]:
def order_peaks_valleys_debug(df, column='Close'):
    '''Remove smallest peak if peaks not followed by valleys & vice versa'''
    
    df = df.copy()
    X = df[column]
    P = argrelextrema(X.to_numpy(), np.greater)[0].tolist() # peaks
    V = argrelextrema(X.to_numpy(), np.less)[0].tolist() # valleys
    # print(f'dates: {df.index[P]}')
    first = 'P' if P[0] < V[0] else 'V'
    pv = {'P', 'V'}
    second = list(pv - {first})[0]
    lst = P if first == 'P' else V
    olst = P if first != 'P' else V
    df['isPeak'] = 0
    df['isValley'] = 0
    df.loc[:df.index[lst[0]], 'Trend'] = 1 if first == 'P' else -1
    fn = min if first == 'P' else max
    removed = []

    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}')
    print(f'{first}: {lst}')
    print(f'{second}: {olst}')
    
    # Remove smallest peak if peak not followed by valley & vice versa
    for i, (p, v) in enumerate(zip(P, V)):
        a = p if first == 'P' else v
        b = v if first == 'P' else p
        if i < len(lst) - 1:
            c = P[i + 1] if first == 'P' else V[i + 1]
            print(f'a: {a}, b: {b}, c: {c}')
            if c < b:
                val = fn((a, X[a]), (c, X[c]), key=itemgetter(1))[0]
                lst.remove(val)
                removed.append(val)
                print(f'\ni: {i}, ({a}, {X[a]:.2f}), ({c}, {X[c]:.2f})')
                print(f'Removed {first}: {date_format(df.index[val])} ({val}, {X[val]:.2f})')
    
    print(f'\nfirst: {first}, peaks: {len(P):,}, valleys: {len(V):,}')
    print(f'{first}: {lst}')
    print(f'{second}: {olst}')
    print(f'removed: {removed}')
    
    return P, V, removed, df

In [11]:
def order_peaks_valleys(df, column='Close'):
    '''Remove smallest peak if peak not followed by valley & vice versa'''
    
    df = df.copy()
    X = df[column]
    P = argrelextrema(X.to_numpy(), np.greater)[0].tolist() # get peaks
    V = argrelextrema(X.to_numpy(), np.less)[0].tolist() # get valleys
    first = 'P' if P[0] < V[0] else 'V'
    lst = P if first == 'P' else V
    # lowest peak if first is 'P', or highest valley if first is 'V'
    fn = min if first == 'P' else max
    removed = []
    
    for i, (p, v) in enumerate(zip(P, V)):
        a = p if first == 'P' else v
        b = v if first == 'P' else p
        if i < len(lst) - 1:
            c = P[i + 1] if first == 'P' else V[i + 1]
            # if peak follows peak or vice versa
            if c < b:
                # value of lowest peak or highest valley
                val = fn((a, X[a]), (c, X[c]), key=itemgetter(1))[0]
                lst.remove(val)
                removed.append(val)
    
    return P, V, removed

In [12]:
ticker = 'mmm'
period = 'D1'
days = TA_PERIODS[period]['days']
end = last_date
start = last_date - timedelta(days)
end = end + timedelta(1) if period.endswith('m') else end
df = make_dataframe(ticker, period=period)[start:end]

P, V , r_PV, pv_df = order_peaks_valleys_debug(df)
pv_df.head()

first: V, peaks: 35, valleys: 35
V: [2, 4, 7, 9, 11, 14, 23, 27, 30, 35, 38, 41, 43, 48, 50, 52, 55, 63, 67, 72, 75, 77, 80, 82, 87, 90, 96, 99, 102, 104, 107, 110, 114, 117, 121]
P: [3, 5, 8, 10, 13, 20, 25, 29, 33, 36, 40, 42, 44, 49, 51, 53, 56, 62, 64, 71, 73, 76, 79, 81, 83, 88, 92, 97, 100, 103, 105, 108, 111, 116, 119]
a: 2, b: 3, c: 4
a: 4, b: 5, c: 7
a: 7, b: 8, c: 9
a: 9, b: 10, c: 11
a: 11, b: 13, c: 14
a: 14, b: 20, c: 23
a: 23, b: 25, c: 27
a: 27, b: 29, c: 30
a: 30, b: 33, c: 35
a: 35, b: 36, c: 38
a: 38, b: 40, c: 41
a: 41, b: 42, c: 43
a: 43, b: 44, c: 48
a: 48, b: 49, c: 50
a: 50, b: 51, c: 52
a: 52, b: 53, c: 55
a: 55, b: 56, c: 63
a: 63, b: 62, c: 67
a: 67, b: 64, c: 72
a: 72, b: 71, c: 75
a: 75, b: 73, c: 77
a: 77, b: 76, c: 80
a: 80, b: 79, c: 82
a: 82, b: 81, c: 87
a: 87, b: 83, c: 90
a: 90, b: 88, c: 96
a: 96, b: 92, c: 99
a: 99, b: 97, c: 102
a: 102, b: 100, c: 104
a: 104, b: 103, c: 107
a: 107, b: 105, c: 110
a: 110, b: 108, c: 114
a: 114, b: 111, c: 117
a: 117

,Open,High,Low,Close,Adj Close,Volume,isPeak,isValley,Trend
Date,,,,,,,,,
2022-12-14,126.5000,128.4700,124.8400,126.0900,122.5790,3095700,0,0,-1.0000
2022-12-15,124.7900,125.0800,121.7500,122.3800,118.9724,3336500,0,0,-1.0000
2022-12-16,121.3100,121.8000,120.0100,121.6900,118.3016,7599800,0,0,-1.0000
2022-12-19,121.3300,123.7700,121.1600,122.1300,118.7293,2440100,0,0,NaN
2022-12-20,121.9900,122.3300,120.7000,120.8100,117.4461,2903000,0,0,NaN


#### Valid Peaks / Valleys

In [13]:
def isConsolidating_debug(X, PV, i):
    n = len(PV)
    x = i
    y = i + 1
    vals = []
    td = X.index[1] - X.index[0]
    
    # percentage variability in prices in a consolidation
    if td.days >= 28 and td.days <= 31:
        pct = 0.1
    if td.days == 7:
        pct = 0.05
    if td.days == 1:
        pct = 0.01
    if td.days == 0:
        pct = 0.005 # 4 hr allowance
        minute = 60
        if td.total_seconds() == 30 * minute:
            pct /= 8
        if td.total_seconds() == 10 * minute:
            pct /= (8 * 3)
        if td.total_seconds() == 5 * minute:
            pct /= (8 * 3 * 2)
        if td.total_seconds() == 1 * minute:
            pct /= (8 * 3 * 2 * 5)
    
    for a in range(i + 2, n, 2):
        b = a + 1
        try:
            v1 = X[PV[a]] / X[PV[x]] - 1 
            v2 = X[PV[b]] / X[PV[y]] - 1 
            line = abs(v1) <= pct and abs(v2) <= pct # values within trading line
            print(f'idxs: [x: {x}, a: {a}], [y: {y}, b: {b}] \
                \ndates: [{date_format(df.index[PV[x]])}, {date_format(df.index[PV[a]])}] - ' 
                f'[{date_format(df.index[PV[y]])}, {date_format(df.index[PV[b]])}] \
                \nv1 = X[PV[a]] / X[PV[x]] - 1 \
                \nv1 = {X[PV[a]]:.2f} / {X[PV[x]]:.2f} - 1 = {v1:.4f} \
                \nv2 = X[PV[b]] / X[PV[y]] - 1 \
                \nv2 = {X[PV[b]]:.2f} / {X[PV[y]]:.2f} - 1 = {v2:.4f} \
                \npct: {pct}')
            if line:
                print('Consolidating...\n')
                vals.extend([PV[x], PV[y]])
                x = a
                y = b
            else:
                if vals and abs(v1) <= pct:
                    vals.extend([PV[x], PV[y], PV[a]])
                    print(f'End consolidation at {date_format(df.index[PV[a]])}')
                print(f'STOP \n\nvals: {vals}')
                break
        except IndexError:
            break

    return vals


In [14]:
def isConsolidating(X, PV, i):
    n = len(PV)
    x = i
    y = i + 1
    vals = []
    td = X.index[1] - X.index[0]
    
    # percentage variability in prices in a consolidation
    if td.days >= 28 and td.days <= 31:
        pct = 0.1
    if td.days == 7:
        pct = 0.05
    if td.days == 1:
        pct = 0.01
    if td.days == 0:
        pct = 0.005 # 4 hr allowance
        minute = 60
        if td.total_seconds() == 30 * minute:
            pct /= 8
        if td.total_seconds() == 10 * minute:
            pct /= (8 * 3)
        if td.total_seconds() == 5 * minute:
            pct /= (8 * 3 * 2)
        if td.total_seconds() == 1 * minute:
            pct /= (8 * 3 * 2 * 5)
    
    for a in range(i + 2, n, 2):
        b = a + 1
        try:
            # v1 (v2) compares distance between current and next peak (valley)
            v1 = X[PV[a]] / X[PV[x]] - 1
            v2 = X[PV[b]] / X[PV[y]] - 1 
            line = abs(v1) <= pct and abs(v2) <= pct # check if values within trading line
            if line:
                vals.extend([PV[x], PV[y]])
                x = a
                y = b
            else:
                if vals and abs(v1) <= pct:
                    vals.extend([PV[x], PV[y], PV[a]])
                break
        except IndexError:
            break

    return vals

In [15]:
def valid_peaks_valleys_debug(df, column='Close', r_intvl=0.05):
    '''
    Remove peaks/valleys with retracement < 1/3

    Parameters
    ----------
    df: DataFrame of a security's market data
    '''
    
    print('S: valid_peaks_valleys'.upper())
    # df = df.copy()
    X = df[column]
    P, V, _ = order_peaks_valleys(df)
    PV = sorted(P + V)
    n = len(PV)
    a, b, c = PV[0], PV[1], PV[2]
    d1 = X[b] - X[a]
    d2 = X[c] - X[b]
    first = 'P' if P[0] < V[0] else 'V'
    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}\n')
    lst = P if first == 'P' else V
    pv = {'P', 'V'}
    second = list(pv - {first})[0]
    third = first if c in lst else list(pv - {first})[0]
    valid = [(a, first), (b, second), (c, third)]
    removed_vals = []
    C = [] # consolidation values  
    retracement = 1/3 * (1 - r_intvl)
    i = 2
    
    while i < n - 1:   
        x = first if c in lst else second
        y = list(pv - {x})[0] 
        r = abs(d2 / d1)
        
        print(f'{i}. {date_format(df.index[b])} - {date_format(df.index[c])} \
              \nc: ({x}, {X[c]:.2f})')
        print('-' * 30)
        print(f'd1: {X[b]:.2f} - {X[a]:.2f} = {d1:.2f}')
        print(f'd2: {X[c]:.2f} - {X[b]:.2f} = {d2:.2f}')
        print(f'r: |{d2:.2f} / {d1:.2f}| = {r:.2f}\n')
        
        last, px = valid[-1]

        if r < retracement and i > 2:
            # NEED TO DEAL WITH RANGING PEAKS/VALLEYS
                        
            ###########################
            ranging = isConsolidating_debug(X, PV, i) if i < n - 4 else False
            if ranging:
                z = ranging[-1]
                td1 = pd.Timedelta(df.index[c] - df.index[b])
                td2 = pd.Timedelta(df.index[z] - df.index[c])
                rtd = td2 / td1
                print(f'[t1: {date_format(df.index[a])}, t2: {date_format(df.index[b])}, ' \
                      f't3: {date_format(df.index[c])}, t4: {date_format(df.index[z])}] \
                      \ntd1: {td1} \ntd2: {td2} \nrtd: {rtd:.2f}')
                if rtd >= retracement:
                    C.append(ranging)
                    for j, val in enumerate(ranging):
                        xtrema = x if j % 2 == 0 else y
                        valid.append((val, xtrema))
                        print(f'Added {date_format(df.index[val])} - {X[val]:.2f} {xtrema}')
                    print(f'i += len(ranging) \n{i} + {len(ranging)} = {i + len(ranging)}')
                    i += len(ranging)
                else:
                    i += 1
            else:
            ###########################

                removed_vals.append((c, x))
                
                # makes first value in list lowest peak / highest valley
                reverse = True if y == 'V' else False
                fn = min if y == 'V' else max
                val = sorted([(b, X[b]), (d, X[d])],
                            key=itemgetter(1), reverse=reverse)
                removed, kept = val[0][0], val[1][0]
                kept1 = fn([(b, X[b]), (d, X[d])], key=itemgetter(1))[0]
                
                print(f'REMOVED: \nvalid[-1]: {px} {date_format(df.index[last])}, ({last}, {X[last]:.2f})')
                print(f'c - {x}, {date_format(df.index[c])} ({c}, {X[c]:.2f})' \
                    f'\nremoved - {y}, {date_format(df.index[removed])} ({removed}, {X[removed]:.2f})' \
                    f'\nkept - {y}, {date_format(df.index[kept])} ({kept}, {X[kept]:.2f})' \
                    f'\nkept1 - {y}, {date_format(df.index[kept1])} ({kept1}, {X[kept1]:.2f})')
                removed_vals.append((removed, y))
                # print(val)
                # valid.append((kept, y))
                if kept == d:
                    i = PV.index(kept)
                    if len(valid) > 2:
                        valid.pop()
                    if i == n - 1:
                        valid.append((kept, y))
                    print(f'\nSkip \ni: {i} c: {kept} {date_format(df.index[kept])} ({y}, {X[kept]:.2f})')
                    # l1 = [PV[i - 2], PV[i - 1], PV[i], PV[i + 1]]
                    # l2 = [valid[-2][0], valid[-1][0], PV[i], PV[i + 1]]
                    # print(f'valid: {l2}\norign: {l1}\nvalid {close[l2]}\norig {close[l1]}\n')

                # Moves to next peak/valley by skipping next value in PV   
                elif kept == last and px == y:
                    print(f'\nUnchanged {(kept, y)}')
                    i += 2
                    print(f'Skip \ni: {i} c: {kept} {date_format(df.index[kept])} ({y}, {X[kept]:.2f})')
                # need to confirm if this is correct
                elif kept == last and px != y:
                    print(f'\nUnchanged 2 {(kept, y)}')
                    i += 1
                else:
                    print(f'DAFUQ!')
                    i += 1
                print()
        else:
            if last != c and px != x:
                valid.append((c, x))
            i += 1    
        
        valid_vals = [(x, round(X[x], 2), y) for x, y in valid]
        print(f'valid[-3:]: {valid_vals[-3:]}\n')
        
        try:
            a, b, c, d = valid[-2][0], valid[-1][0], PV[i], PV[i + 1]
            d1 = abs(X[b] - X[a])
            d2 = abs(X[c] - X[b])
        # Add last peak/valley if it's valid
        except Exception as e:
            if i == n - 1:
                a, b, c = valid[-2][0], valid[-1][0], PV[i]
                d1 = abs(X[b] - X[a])
                d2 = abs(X[c] - X[b])
                r = d2 / d1
                if r >= retracement:
                    valid.append((c, y))
                print(f'LAST {i} {df.index[c]} \nc: ({y}, {X[c]:.2f})')
            else:
                print(e)

    print(f'final valid: {valid}')
    
    P, V = [], []
    C_ = [x[-1] for x in C]

    for val, x in valid:
        lst = P if x == 'P' else V
        lst.append(val)

    PV = P + V

    for x in C:
        for y in x:
            if y not in PV:
                print(f'removing {y} from c_vals')
                x.remove(y)

    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}\n')
    print('E: valid_peaks_valleys'.upper())

    return P, V, C

In [16]:
def valid_peaks_valleys(df, column='Close', r_intvl=0.05):
    '''
    Remove peaks/valleys with retracement < 1/3

    Parameters
    ----------
    df: DataFrame of a security's market data
    column: column in DataFrame
    r_intvl: confidence interval for retracement
    '''

    X = df[column]
    P, V, _ = order_peaks_valleys(df, column=column)
    PV = sorted(P + V)
    n = len(PV)
    a, b, c = PV[0], PV[1], PV[2]
    d1 = abs(X[b] - X[a])
    d2 = abs(X[c] - X[b])
    first = 'P' if P[0] < V[0] else 'V'
    lst = P if first == 'P' else V
    pv = {'P', 'V'}
    second = list(pv - {first})[0]
    third = first if c in lst else second
    valid = [(a, first), (b, second), (c, third)]
    retracement = 1 / 3 * (1 - r_intvl)
    i = 2
    
    while i < n - 1:   
        x = first if c in lst else second
        y = list(pv - {x})[0] 
        r = d2 / d1
        last, px = valid[-1]
        
        if r < retracement and i > 2:
            ranging = isConsolidating(X, PV, i) if i < n - 4 else False
            if ranging:
                z = ranging[-1]
                td1 = pd.Timedelta(df.index[c] - df.index[b])
                td2 = pd.Timedelta(df.index[z] - df.index[c])
                rtd = td2 / td1
                if rtd >= retracement:
                    for j, val in enumerate(ranging):
                        xtrema = x if j % 2 == 0 else y
                        valid.append((val, xtrema))
                    i += len(ranging)
                else:
                    i += 1
            else:
                fn = min if y == 'V' else max
                kept = fn([(b, X[b]), (d, X[d])], key=itemgetter(1))[0]
                if kept == d:
                    i = PV.index(kept)
                    if len(valid) > 2:
                        valid.pop()
                    if i == n - 1:
                        valid.append((kept, y))
                elif kept == last and px == y:
                    i += 2
                else:
                    i += 1
        else:
            if last != c and px != x:
                valid.append((c, x))
            i += 1  
              
        try:
            a, b, c, d = valid[-2][0], valid[-1][0], PV[i], PV[i + 1]
            d1 = abs(X[b] - X[a])
            d2 = abs(X[c] - X[b])
        except Exception as e:
            if i == n - 1:
                a, b, c = valid[-2][0], valid[-1][0], PV[i]
                d1 = abs(X[b] - X[a])
                d2 = abs(X[c] - X[b])
                r = d2 / d1
                if r >= retracement:
                    valid.append((c,y))
            else:
                print(e)
              
    P, V = [], []
    
    for val, x in valid:
        lst = P if x == 'P' else V
        lst.append(val)

    return P, V

In [17]:
period = 'W1'
days = TA_PERIODS[period]['days']
end = last_date
start = end - timedelta(days)
ticker = 'cf' 
df = make_dataframe(ticker, period)[start:end]
print(ticker.upper(), '-', period)
P, V, C = valid_peaks_valleys_debug(df)

CF - W1
S: VALID_PEAKS_VALLEYS
first: P, peaks: 24, valleys: 24

2. 16/07/21 - 23/07/21               
c: (P, 47.31)
------------------------------
d1: 47.16 - 51.59 = -4.43
d2: 47.31 - 47.16 = 0.15
r: |0.15 / -4.43| = 0.03

valid[-3:]: [(2, 51.59, 'P'), (4, 47.16, 'V'), (5, 47.31, 'P')]

3. 23/07/21 - 06/08/21               
c: (V, 46.19)
------------------------------
d1: 47.31 - 47.16 = 0.15
d2: 46.19 - 47.31 = 1.12
r: |1.12 / 0.15| = 7.47

valid[-3:]: [(4, 47.16, 'V'), (5, 47.31, 'P'), (7, 46.19, 'V')]

4. 06/08/21 - 13/08/21               
c: (P, 48.38)
------------------------------
d1: 46.19 - 47.31 = 1.12
d2: 48.38 - 46.19 = 2.19
r: |2.19 / 1.12| = 1.96

valid[-3:]: [(5, 47.31, 'P'), (7, 46.19, 'V'), (8, 48.38, 'P')]

5. 13/08/21 - 20/08/21               
c: (V, 44.47)
------------------------------
d1: 48.38 - 46.19 = 2.19
d2: 44.47 - 48.38 = 3.91
r: |3.91 / 2.19| = 1.79

valid[-3:]: [(7, 46.19, 'V'), (8, 48.38, 'P'), (9, 44.47, 'V')]

6. 20/08/21 - 01/10/21               
c: 

#### trend_changepoints_debug

In [18]:
def trend_changepoints_debug_2(df, column='Close'):
    X = df[column]
    chg = {k: {'begin': [], 'end': []} 
           for k in ['up', 'down', 'ranging']} # trend changepoints
    t = None # trend
    P, V = valid_peaks_valleys(df)
    PV = sorted(P + V)
    first = 'P' if P[0] < V[0] else 'V'
    up, down, ranging = [], [], []
    C_ = [y for x in C for y in [x[0], x[-1]]] # first and last values in each consolidation period
    C = [y for x in C for y in x]

    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}')
    
    ####################################
    i = 2
    n = min(len(P), len(V))
    

    # for i, (p, v) in enumerate(zip(P[2:], V[2:]), 2):
    while i < n - 1:
        p, v = P[i], V[i] ##
        g, h = i - 2, i - 1
        # Setup of dataframe for debugging
        try:
            j, a, b, c = P[g - 1], P[g], P[h], p
            k, d, e, f = V[g - 1], V[g], V[h], v
            p00, p0, p1, p2, v00, v0, v1, v2 = X[[j,a,b,c,k,d,e,f]]
            curr_p, curr_v = [p00, p0, p1, p2], [v00, v0, v1, v2]
            p_dates = df.index[[j,a,b,c]].to_list()
            v_dates = df.index[[k,d,e,f]].to_list()
        except:
            a, b, c = P[g], P[h], p
            d, e, f = V[g], V[h], v
            p0, p1, p2, v0, v1, v2 = X[[a,b,c,d,e,f]]
            curr_p, curr_v = [p0, p1, p2], [v0, v1, v2]
            p_dates = df.index[[a,b,c]].to_list()
            v_dates = df.index[[d,e,f]].to_list()
        dates = sorted([p, v])
        dates = [date_format(df.index[x]) for x in dates]
        print(f'\n{i}. {dates}')
        if t: 
            print(f'Prev. Trend: {t.upper()}')
        else:
            print(f'Prev. Trend: {t}')
        print(f'p0: {p0:.2f}, p1: {p1:.2f}, p2: {p2:.2f} \
              \nv0: {v0:.2f}, v1: {v1:.2f}, v2: {v2:.2f}')
        p_data = {'Peaks': curr_p}
        v_data = {'Valleys': curr_v}
        p_df = pd.DataFrame(p_data, index=p_dates)
        v_df = pd.DataFrame(v_data, index=v_dates)
        pv_df = pd.concat([p_df, v_df]).sort_index().fillna('-')
        pv_df.index = [date_format(x) for x in pv_df.index]
        st_txt = '-' * 30 + '\nStart'

        if first == 'P':
            uptrend = p2 > p1 and v1 > v0
            downtrend = v1 < v0 and p2 < p1
        else: 
            uptrend = p2 > p1 and v2 > v1
            downtrend = v2 < v1 and p1 < p0

        consolidation = True if p in C and v in C else False ##

        if uptrend:
            c_ix = p
            print(f'Trend: Uptrend')
            if first == 'P':
                print('p2 > p1 and v1 > v0')
            else:
                print('p2 > p1 and v2 > v1')

            if t is None:
                chg['up']['begin'].append(d)
                up.append(d)
            elif t != 'up':
                ix = d if first == 'P' else e
                print(f'End {t.upper()} - {ix} - ({date_format(df.index[ix])}, {X[ix]:.2f})')
                print(f'{st_txt} {date_format(df.index[ix])}\n')
                if chg[t]['begin']:
                    chg[t]['end'].append(ix)
                t = 'up'
                chg[t]['begin'].append(ix)
                up.append(ix)
                if consolidation:
                    chg[t]['end'].append(c_ix)
                    t = 'ranging'
                    chg[t]['begin'].append(c_ix)
                    ranging.append(c_ix)
                    idx = bisect.bisect(C_, c_ix)
                    x0 = C_[idx]
                    chg[t]['end'].append(x0)
                    i = PV.index(x0)
                    print(f'Trend: Consolidating \
                        \np, v: ({p}, {v}) [{date_format(df.index[p])}, {date_format(df.index[v])}] \
                        \nConsolidation END: {c_ix} {date_format(df.index[x0])} - {X[x0]:.2f}')
                pv_df['Trend'] = [t] * pv_df.shape[0]
                print(f'{pv_df}\n')
                pprint(chg)
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[c])} - {X[c]:.2f}')
                pprint(chg)
            t = 'up'
        elif downtrend:
            c_ix = v
            print(f'Trend: Downtrend')
            if first == 'P':
                print('v1 < v0 and p2 < p1')
            else:
                print('v2 < v1 and p1 < p0')
            
            if t is None:
                chg['down']['begin'].append(a)
                down.append(a)
            elif t != 'down':
                ix = a if first == 'V' else b
                print(f'End {t.upper()} - {ix} - ({date_format(df.index[ix])}, {X[ix]:.2f})')
                print(f'{st_txt} {date_format(df.index[ix])}\n')
                if chg[t]['begin']:
                    chg[t]['end'].append(ix)
                t = 'down'
                chg[t]['begin'].append(ix)
                down.append(ix)
                if consolidation:
                    chg[t]['end'].append(c_ix)
                    t = 'ranging'
                    chg[t]['begin'].append(c_ix)
                    ranging.append(c_ix)
                    idx = bisect.bisect(C_, c_ix)
                    x0 = C_[idx]
                    chg[t]['end'].append(x0)
                    i = PV.index(x0)
                    print(f'Trend: Consolidating \
                        \np, v: ({p}, {v}) [{date_format(df.index[p])}, {date_format(df.index[v])}] \
                        \nConsolidation END: {c_ix} {date_format(df.index[x0])} - {X[x0]:.2f}')
                
                pv_df['Trend'] = [t] * pv_df.shape[0]
                print(f'{pv_df}\n')
                pprint(chg)
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[f])} - {X[f]:.2f}')
                pprint(chg)
            t = 'down'
        else:
            print(f'Trend: Ranging')
            if t is None:
                ix = a if first == 'V' else d
                chg['ranging']['begin'].append(ix)
                ranging.append(ix)
                # t = 'ranging'
            elif t != 'ranging': # Ranging
                if t == 'up':
                    ix = a if first == 'P' else b
                    c_ix = p ##
                    cont = p2 > p1 
                else:
                    ix = d if first == 'P' else e ##
                    c_ix = v ##
                    cont = v1 < v0 if first == 'P' else v2 < v1
                #################
                # if cont:
                #     print(f'Continue {t.upper()}')
                # else:
                if chg[t]['begin']:
                    chg[t]['end'].append(ix)
                    print(f'End {t.upper()} - {ix} - ({date_format(df.index[ix])}, {X[ix]:.2f})\n')
                t = 'ranging'
                chg[t]['begin'].append(ix)
                ranging.append(ix)
                if consolidation:
                    chg[t]['end'].append(c_ix)
                    t = 'ranging'
                    chg[t]['begin'].append(c_ix)
                    ranging.append(c_ix)
                    idx = bisect.bisect(C_, c_ix)
                    x0 = C_[idx]
                    chg[t]['end'].append(x0)
                    i = PV.index(x0)
                    print(f'Trend: Consolidating \
                        \np, v: ({p}, {v}) [{date_format(df.index[p])}, {date_format(df.index[v])}] \
                        \nConsolidation END: {c_ix} {date_format(df.index[x0])} - {X[x0]:.2f}')
                
                print(f'Begin {t.upper()} - {ix} - ({date_format(df.index[ix])}, {X[ix]:.2f})\n')
                #################
                
                pv_df['Trend'] = [t] * pv_df.shape[0]
                print(f'{pv_df}\n')
                pprint(chg)
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[f])} - {X[f]:.2f}')
            t = 'ranging'  
        i += 1
    
    print('\nChangepoints:')
    pprint(chg)
    
    dr = chg['ranging']

    for i, (x, y) in enumerate(zip(dr['begin'], dr['end'])):
        if x == y:
            print(f'\nRemoved {x}')
            dr['begin'].pop(i)
            dr['end'].pop(i)
        
    # Add last value in data to appropriate changepoint
    print(f'\nup: {up}, down: {down}, ranging: {ranging}')
    x = sorted(up + down + ranging)[-1]
    print(f'Last Value: {x}\n')
    
    for k, v in chg.items():
        if x in v['begin']:
            # print(k1, v1)
            print(f"{k}['begin']: {v['begin']} \n{k}['end']: {v['end']}")
            v['end'].append(df.shape[0] - 1)
            print(f'\nLast val - {df.shape[0] - 1} in dict("{k}")\n')
            break
    
    print('Final changepoints:')
    pprint(chg)
    
    return chg

In [19]:
def trend_changepoints_debug(df, column='Close'):
    X = df[column]
    chg = {k: {'begin': [], 'end': []} 
           for k in ['up', 'down', 'ranging']} # trend changepoints
    t = None # trend
    P, V = valid_peaks_valleys(df)
    first = 'P' if P[0] < V[0] else 'V'
    lst, n = min((P, len(P)), (V, len(V)), key=itemgetter(1))
    fill = lst[-1]

    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}')
    
    # Identify trend changepoints 
    for i, (p, v) in enumerate(zip_longest(P[2:], V[2:], fillvalue=fill), 2):
        g, h = i - 2, i - 1        
        a, b, c, d, e, f = P[g], P[h], p, V[g], V[h], v
        p0, p1, p2, v0, v1, v2 = X[[a,b,c,d,e,f]]

        # Setup of dataframe for debugging
        try:
            j, a, b, c = P[g - 1], P[g], P[h], p
            k, d, e, f = V[g - 1], V[g], V[h], v
            p00, p0, p1, p2, v00, v0, v1, v2 = X[[j,a,b,c,k,d,e,f]]
            curr_p, curr_v = [p00, p0, p1, p2], [v00, v0, v1, v2]
            p_dates = df.index[[j,a,b,c]].to_list()
            v_dates = df.index[[k,d,e,f]].to_list()
        except:
            a, b, c = P[g], P[h], p
            d, e, f = V[g], V[h], v
            p0, p1, p2, v0, v1, v2 = X[[a,b,c,d,e,f]]
            curr_p, curr_v = [p0, p1, p2], [v0, v1, v2]
            p_dates = df.index[[a,b,c]].to_list()
            v_dates = df.index[[d,e,f]].to_list()

        dates = sorted([p, v])
        dates = [date_format(df.index[x]) for x in dates]
        print(f'\n{i}. {dates}')
        if t: 
            print(f'Prev. Trend: {t.upper()}')
        else:
            print(f'Prev. Trend: {t}')
        print(f'p0: {p0:.2f}, p1: {p1:.2f}, p2: {p2:.2f} \
              \nv0: {v0:.2f}, v1: {v1:.2f}, v2: {v2:.2f}')
        p_data = {'Peaks': curr_p}
        v_data = {'Valleys': curr_v}
        p_df = pd.DataFrame(p_data, index=p_dates)
        v_df = pd.DataFrame(v_data, index=v_dates)
        pv_df = pd.concat([p_df, v_df]).sort_index().fillna('-')
        pv_df.index = [date_format(x) for x in pv_df.index]
        st_txt = '-' * 30 + '\nStart'

        if first == 'P':
            uptrend = p2 > p1 and v1 > v0
            downtrend = v1 < v0 and p2 < p1
        else: 
            uptrend = p2 > p1 and v2 > v1
            downtrend = v2 < v1 and p1 < p0
        
        ranging_to_down = p2 < p1 and v2 < v1
        ranging_to_up = p2 > p1 and v2 > v1
        ranging_to = ranging_to_down or ranging_to_up

        if uptrend:
            print(f'Trend: UPTREND')
            if first == 'P':
                print('p2 > p1 and v1 > v0')
            else:
                print('p2 > p1 and v2 > v1')

            if t is None:
                t = 'up'
                ix = d
                chg[t]['begin'].append(ix)
            elif t != 'up':
                ix = d if first == 'P' else e
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[c])} - {X[c]:.2f}')
            ct = 'up'
        elif downtrend:
            print(f'Trend: DOWNTREND')
            if first == 'P':
                print('v1 < v0 and p2 < p1')
            else:
                print('v2 < v1 and p1 < p0')
            
            if t is None:
                t = 'down'
                ix = a
                chg[t]['begin'].append(ix)
            elif t != 'down':
                ix = a if first == 'V' else b
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[f])} - {X[f]:.2f}')
            ct = 'down'
        else:
            print(f'Trend: RANGING')
            if t is None:
                ix = a if first == 'P' else d
                t = 'ranging'
                chg[t]['begin'].append(ix)
            elif t != 'ranging':
                ix = b if t == 'up' else e
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[f])} - {X[f]:.2f}')
            ct = 'ranging'
        
        pv_df['Trend'] = [ct] * pv_df.shape[0]
        print(f'{pv_df}\n')
        pprint(chg)
            

        print(f'{st_txt} {date_format(df.index[ix])}\n')
        if chg[t]['begin']:
            if chg[t]['begin'][-1] != ix:
                chg[t]['end'].append(ix)
                print(f'End {t.upper()} - {ix} - ({date_format(df.index[ix])}, {X[ix]:.2f})\n')
            else:
                chg[t]['begin'].pop()
                print(f'Removed from {t}[begin]: ({ix} - {date_format(df.index[ix])})\n')

        chg[ct]['begin'].append(ix)
        t = ct            

        if i == n:
            if uptrend:
                ix = p
            elif downtrend:
                ix = v
            else:
                ix = max(p, v)
            chg[t]['end'].append(ix)
            print(f'{t}[end] = {ix}')
    
    print('\nChangepoints:')
    pprint(chg)
    print()
    
    last = max([chg[k]['end'][-1] for k in chg.keys()])
    
    for k, v in chg.items():
        if last in v['begin']:
            if k == 'up':
                ix = max((P[-1], X[P[-1]]), (P[-2], X[P[-2]]), 
                         key=itemgetter(1))[0]
            elif k == 'down':
                ix = min((V[-1], X[V[-1]]), (V[-2], X[V[-2]]), 
                         key=itemgetter(1))[0]
            else:
                ix = max(P[-1], V[-1])
            v['end'].append(ix)
            print(f"{k}['end']: {v['end'][-1]} - ({date_format(df.index[v['end'][-1]])}, {X[v['end'][-1]]:.2f})")     
            print(f'\nLast val - {df.shape[0] - 1} in dict("{k}")\n')
    
    print('Final changepoints:')
    pprint(chg)
    
    return chg

In [20]:
def trend_changepoints_debug_1(df, column='Close'):
    X = df[column]
    chg = {k: {'begin': [], 'end': []} 
           for k in ['up', 'down', 'ranging']} # trend changepoints
    t = None # trend
    P, V = valid_peaks_valleys(df)
    first = 'P' if P[0] < V[0] else 'V'
    lst, n = min((P, len(P)), (V, len(V)), key=itemgetter(1))
    fill = lst[-1]

    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}')
    
    # Identify trend changepoints 
    for i, (p, v) in enumerate(zip_longest(P[2:], V[2:], fillvalue=fill), 2):
        g, h = i - 2, i - 1
        a, b, c, d, e, f = P[g], P[h], p, V[g], V[h], v ##
        p0, p1, p2, v0, v1, v2 = X[[a,b,c,d,e,f]]

        # Setup of dataframe for debugging
        try:
            j, a, b, c = P[g - 1], P[g], P[h], p
            k, d, e, f = V[g - 1], V[g], V[h], v
            p00, p0, p1, p2, v00, v0, v1, v2 = X[[j,a,b,c,k,d,e,f]]
            curr_p, curr_v = [p00, p0, p1, p2], [v00, v0, v1, v2]
            p_dates = df.index[[j,a,b,c]].to_list()
            v_dates = df.index[[k,d,e,f]].to_list()
        except:
            a, b, c = P[g], P[h], p
            d, e, f = V[g], V[h], v
            p0, p1, p2, v0, v1, v2 = X[[a,b,c,d,e,f]]
            curr_p, curr_v = [p0, p1, p2], [v0, v1, v2]
            p_dates = df.index[[a,b,c]].to_list()
            v_dates = df.index[[d,e,f]].to_list()

        dates = sorted([p, v])
        dates = [date_format(df.index[x]) for x in dates]
        print(f'\n{i}. {dates}')

        if t: 
            print(f'Prev. Trend: {t.upper()}')
        else:
            print(f'Prev. Trend: {t}')

        print(f'p0: {p0:.2f}, p1: {p1:.2f}, p2: {p2:.2f} \
              \nv0: {v0:.2f}, v1: {v1:.2f}, v2: {v2:.2f}')
        
        p_data = {'Peaks': curr_p}
        v_data = {'Valleys': curr_v}
        p_df = pd.DataFrame(p_data, index=p_dates)
        v_df = pd.DataFrame(v_data, index=v_dates)
        pv_df = pd.concat([p_df, v_df]).sort_index().fillna('-')
        pv_df.index = [date_format(x) for x in pv_df.index]
        st_txt = '-' * 30 + '\nStart'

        if first == 'P':
            uptrend = p2 > p1 and v1 > v0
            downtrend = v1 < v0 and p2 < p1
        else: 
            uptrend = p2 > p1 and v2 > v1
            downtrend = v2 < v1 and p1 < p0
        
        ranging_to_down = p2 < p1 and v2 < v1
        ranging_to_up = p2 > p1 and v2 > v1
        ranging_to = ranging_to_down or ranging_to_up

        if uptrend:
            print(f'Trend: UPTREND')
            if first == 'P':
                print('p2 > p1 and v1 > v0')
            else:
                print('p2 > p1 and v2 > v1')

            if t is None:
                ix = d
            elif t != 'up':
                ix = d if first == 'P' else e
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[c])} - {X[c]:.2f}')
            ct = 'up'   
        elif downtrend:
            print(f'Trend: DOWNTREND')
            if first == 'P':
                print('v1 < v0 and p2 < p1')
            else:
                print('v2 < v1 and p1 < p0')
            
            if t is None:
                ix = a
            elif t != 'down':
                ix = a if first == 'V' else b
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[f])} - {X[f]:.2f}')
            ct = 'down'
        else:
            print(f'Trend: RANGING')
            if t is None:
                ix = a if first == 'P' else d
            elif t != 'ranging':
                if t == 'up':
                    ix = b if first == 'V' else c
                else:
                    ix = e if first == 'P' else f
            else:
                print(f'Continue {t.upper()}')
                print(f'{date_format(df.index[f])} - {X[f]:.2f}')
            ct = 'ranging'
            
        print(f'{st_txt} {date_format(df.index[ix])}\n')
        if t and chg[t]['begin']:
            if chg[t]['begin'][-1] != ix:
                chg[t]['end'].append(ix)
                print(f'End {t.upper()}: {ix} - ({date_format(df.index[ix])}, {X[ix]:.2f})\n')
            else:
                chg[t]['begin'].pop()
                print(f'Removed from {t}[begin]: {ix} - {date_format(df.index[ix])}')
       
        chg[ct]['begin'].append(ix)
        t = ct
        print(f'Added to {t}[begin]: {ix} - {date_format(df.index[ix])}\n')

        pv_df['Trend'] = [ct] * pv_df.shape[0]
        print(f'{pv_df}\n')
        pprint(chg)

        if i == n:
            if uptrend:
                ix = p
            elif downtrend:
                ix = v
            else:
                ix = max(p, v)
            chg[t]['end'].append(ix) 
            print(f'i == n: {t.upper} - max(p, v) = {ix} - ({date_format(df.index[ix])}, {X[ix]:.2f})')
            print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}')
                
    print('\nChangepoints:')
    pprint(chg)
    print()
    
    last = max([chg[k]['end'][-1] for k in chg.keys()])
    
    for k, v in chg.items():
        if last in v['begin']:
            if k == 'up':
                ix = P[-1]
            elif k == 'down':
                ix = V[-1]
            else:
                ix = max(P[-1], V[-1])
            v['end'].append(ix)
            print(f"{k}['begin']: {last} - ({date_format(df.index[last])}, {X[last]:.2f})")
            print(f"{k}['end']: {v['end'][-1]} - ({date_format(df.index[v['end'][-1]])}, {X[v['end'][-1]]:.2f})")     
            print(f'\nLast val - {df.shape[0] - 1} in dict("{k}")\n')
    
    print('Final changepoints:')
    pprint(chg)
    
    return chg

In [21]:
ticker = 'amcr'
print('Ticker:', ticker.upper())
period = 'D1'
days = TA_PERIODS[period]['days']
end = last_date
start = last_date - timedelta(days)
end = end + timedelta(1) if period.endswith('m') else end
df = make_dataframe(ticker, period=period)[start:end]
# P, V, *_ = order_peaks_valleys(df)
# P, V = valid_peaks_valleys(df)
chg = trend_changepoints_debug(df)
# pprint(chg)

Ticker: AMCR
first: V, peaks: 24, valleys: 24

2. ['28/12/22', '29/12/22']
Prev. Trend: None
p0: 12.10, p1: 12.08, p2: 12.02               
v0: 12.02, v1: 11.98, v2: 11.91
Trend: DOWNTREND
v2 < v1 and p1 < p0
           Peaks Valleys Trend
19/12/22       - 12.0200  down
21/12/22 12.1000       -  down
22/12/22       - 11.9800  down
27/12/22 12.0800       -  down
28/12/22       - 11.9100  down
29/12/22 12.0200       -  down
31/05/23       -  9.6400  down
07/06/23 10.1100       -  down

{'down': {'begin': [5], 'end': []},
 'ranging': {'begin': [], 'end': []},
 'up': {'begin': [], 'end': []}}
------------------------------
Start 21/12/22

Removed from down[begin]: (5 - 21/12/22)


3. ['30/12/22', '04/01/23']
Prev. Trend: DOWN
p0: 12.08, p1: 12.02, p2: 12.03               
v0: 11.98, v1: 11.91, v2: 11.91
Trend: RANGING
           Peaks Valleys    Trend
19/12/22       - 12.0200  ranging
21/12/22 12.1000       -  ranging
22/12/22       - 11.9800  ranging
27/12/22 12.0800       -  ranging
28/1

In [22]:
def trend_changepoints(df, column='Close'):
    X = df[column]
    P, V = valid_peaks_valleys(df, column=column)
    chg = {k: {'begin': [], 'end': []} 
           for k in ['up', 'down', 'ranging']} # trend changepoints
    t = None # trend
    first = 'P' if P[0] < V[0] else 'V'
    lst, n = min((P, len(P)), (V, len(V)), key=itemgetter(1))
    fill = lst[-1]
    
    # Identify trend changepoints 
    for i, (p, v) in enumerate(zip_longest(P[2:], V[2:], fillvalue=fill), 2):
        g, h = i - 2, i - 1
        a, b, c, d, e, f = P[g], P[h], p, V[g], V[h], v
        p0, p1, p2, v0, v1, v2 = X[[a,b,c,d,e,f]]
        
        if first == 'P':
            uptrend = p2 > p1 and v1 > v0
            downtrend = v1 < v0 and p2 < p1
        else: 
            uptrend = p2 > p1 and v2 > v1
            downtrend = v2 < v1 and p1 < p0

        # Changes need to be made for handling previous ranging in up/downtrends
        if uptrend:
            if t is None:
                t = 'up'
                ix = d
                chg[t]['begin'].append(ix)
            elif t != 'up':
                ix = d if first == 'P' else e
            ct = 'up'
        elif downtrend:
            if t is None:
                t = 'down'
                ix = a
                chg[t]['begin'].append(ix)
            elif t != 'down':
                ix = a if first == 'V' else b
            ct = 'down'
        else:
            if t is None:
                ix = a if first == 'P' else d
                t = 'ranging'
                chg[t]['begin'].append(ix)
            elif t != 'ranging':
                ix = b if t == 'up' else e
            ct = 'ranging'
        
        if chg[t]['begin']:
            if chg[t]['begin'][-1] != ix:
                chg[t]['end'].append(ix)
            else:
                chg[t]['begin'].pop()
        chg[ct]['begin'].append(ix)
        t = ct
        
        if i == n:
            if uptrend:
                ix = p
            elif downtrend:
                ix = v
            else:
                ix = max(p, v)
            chg[t]['end'].append(ix) 
     
    last = max([chg[k]['end'][-1] for k in chg.keys()])
    
    # Endpoint of last trend
    for k, v in chg.items():
        if last in v['begin']:
            if k == 'up':
                ix = max((P[-1], X[P[-1]]), (P[-2], X[P[-2]]), 
                         key=itemgetter(1))[0]
            elif k == 'down':
                ix = min((V[-1], X[V[-1]]), (V[-2], X[V[-2]]), 
                         key=itemgetter(1))[0]
            else:
                ix = max(P[-1], V[-1])
            v['end'].append(ix) 
    
    return chg


In [23]:
def trend_changepoints_1(df, column='Close'):
    X = df[column]
    P, V = valid_peaks_valleys(df, column=column)
    chg = {k: {'begin': [], 'end': []} 
           for k in ['up', 'down', 'ranging']} # trend changepoints
    t = None # trend
    first = 'P' if P[0] < V[0] else 'V'
    lst, n = min((P, len(P)), (V, len(V)), key=itemgetter(1))
    fill = lst[-1]
    
    # Identify trend changepoints 
    for i, (p, v) in enumerate(zip_longest(P[2:], V[2:], fillvalue=fill), 2):
        g, h = i - 2, i - 1
        a, b, c, d, e, f = P[g], P[h], p, V[g], V[h], v
        p0, p1, p2, v0, v1, v2 = X[[a,b,c,d,e,f]]
        
        if first == 'P':
            uptrend = p2 > p1 and v1 > v0
            downtrend = v1 < v0 and p2 < p1
        else: 
            uptrend = p2 > p1 and v2 > v1
            downtrend = v2 < v1 and p1 < p0

        # Changes need to be made for handling previous ranging in up/downtrends
        if uptrend:
            if t is None:
                t = 'up'
                ix = d
                chg[t]['begin'].append(ix)
            elif t != 'up':
                ix = d if first == 'P' else e
            ct = 'up'
        elif downtrend:
            if t is None:
                t = 'down'
                ix = a
                chg[t]['begin'].append(ix)
            elif t != 'down':
                ix = a if first == 'V' else b
            ct = 'down'
        else:
            if t is None:
                ix = a if first == 'P' else d
                t = 'ranging'
                chg[t]['begin'].append(ix)
            elif t != 'ranging':
                if t == 'up':
                    ix = a if first == 'V' else b
                else:
                    ix = d if first == 'P' else e
            ct = 'ranging'
        
        if chg[t]['begin']:
            if chg[t]['begin'][-1] != ix:
                chg[t]['end'].append(ix)
            else:
                chg[t]['begin'].pop()
        chg[ct]['begin'].append(ix)
        t = ct
        
        if i == n:
            if uptrend:
                ix = p
            elif downtrend:
                ix = v
            else:
                ix = max(p, v)
            chg[t]['end'].append(ix) 
     
    last = max([chg[k]['end'][-1] for k in chg.keys()])
    
    # Endpoint of last trend
    for k, v in chg.items():
        if last in v['begin']:
            if k == 'up':
                ix = P[-1]
            elif k == 'down':
                ix = V[-1]
            else:
                ix = max(P[-1], V[-1])
            v['end'].append(ix) 
    
    return chg


In [24]:
chg = trend_changepoints(df)
chg

{'up': {'begin': [11, 26, 38, 63], 'end': [19, 33, 42, 73]},
 'down': {'begin': [5, 19, 42, 49, 60, 73, 98],
  'end': [9, 26, 45, 58, 63, 95, 114]},
 'ranging': {'begin': [9, 33, 45, 58, 95], 'end': [11, 38, 49, 60, 98]}}

In [25]:
trend_changepoints_1(df)

{'up': {'begin': [11, 26, 38, 63], 'end': [13, 29, 40, 65]},
 'down': {'begin': [5, 19, 42, 49, 60, 73, 98],
  'end': [9, 26, 45, 58, 63, 95, 114]},
 'ranging': {'begin': [9, 13, 29, 40, 45, 58, 65, 95],
  'end': [11, 19, 38, 42, 49, 60, 73, 98]}}

### First == Peak or Valley

In [26]:
def stocks_first_peak_valley():
    '''Stocks classified into those beginning with peaks & valleys'''
    period = 'D1'
    days = TA_PERIODS[period]['days']
    end = last_date
    start = end - timedelta(days)
    # ticker = None
    firsts = set()
    Ps = []
    Vs = []

    for i, x in enumerate(ticker_list):
        print(f'\r{i}', end='', flush=True)
        df = make_dataframe(x, period)[start:end]
        #####################
    #     P, V = valid_peaks_valleys(df)
    #     first = 'P' if P[0] < V[0] else 'V'
    #     if first == 'P':
    #         Ps.append(x)
    #     else:
    #         Vs.append(x)

    # print(f'\nP: {Ps} \nV: {Vs}')
    # return Ps, Vs
########################
        P, V, removed = order_peaks_valleys(df)
        if removed:
            first = 'P' if P[0] < V[0] else 'V'
            firsts.add(first)
            if first == 'P':
                Ps.append(x)
            else:
                Vs.append(x)
            # print(f'{first}: {x}')
            # ticker = x
        if len(firsts) == 2 and len(Ps) >= 3 and len(Vs) >= 3:
            print(f'\nP: {Ps} \nV: {Vs}')
            break
        elif i == len(ticker_list) - 1:
            print(f'\nP: {Ps} \nV: {Vs}')

stocks_first_peak_valley()    

17

178
P: ['BKR', 'CAH', 'EQT'] 
V: ['ACN', 'AMCR', 'AME', 'AON', 'ANET', 'BALL', 'CPB', 'CTLT', 'CDAY', 'CMS', 'DAL', 'DVN', 'D']


In [27]:
ticker = 'acn'
print(ticker.upper())
period = 'D1'
days = TA_PERIODS[period]['days']
end = last_date
start = last_date - timedelta(days)
end = end + timedelta(1) if period.endswith('m') else end
df = make_dataframe(ticker, period=period)[start:end]
# P, V, *_ = order_peaks_valleys(df)
# P, V = valid_peaks_valleys(df)
# chg = trend_changepoints(df)
chg = trend_changepoints(df)
chg

ACN


{'up': {'begin': [27, 99], 'end': [33, 116]},
 'down': {'begin': [5, 33, 40, 81], 'end': [9, 39, 61, 99]},
 'ranging': {'begin': [9, 39, 61, 116], 'end': [27, 40, 81, 120]}}

### Plot Trends

In [46]:
def plot_trends(ticker, period='D1', graph='line', bgcolor='slategray'):
    period_d = TA_PERIODS[period]
    days = period_d['days']
    end = last_date
    start = end - timedelta(days)
    end = end + timedelta(1) if period.endswith('m') else end
    df = make_dataframe(ticker, period)[start:end]
    
    data = []
    nrows = 1
    fig = make_subplots(rows=nrows, cols=1,
                        shared_xaxes=True, 
                        vertical_spacing=0.05,
                        subplot_titles=[''] * nrows, 
                        )

    if graph == 'candlestick':
        cs = go.Candlestick(x=df.index, 
                            open=df['Open'], 
                            high=df['High'],
                            low=df['Low'], 
                            close=df['Close'],
                            name=ticker)
        cs.increasing.fillcolor = 'green'
        cs.increasing.line.color = 'darkgreen'
        cs.decreasing.fillcolor = 'red'
        cs.decreasing.line.color = 'indianred'
        data.append(cs)
    else:   
        line = go.Scatter(x=df.index, 
                          y=df.Close, 
                          name='Close', 
                          mode='lines',
                          line_color='blue', 
                          opacity=0.5, 
                          connectgaps=True)
        data.append(line)

    pos = [1] * len(data) # position to add rows, cols in subplot 
    fig.add_traces(data=data, rows=pos, cols=pos)  
    
    # show all peaks/valleys & removed peaks/valleys
    vals = order_peaks_valleys(df)
    keys = ['peaks', 'valleys', 'removed']
    # pv_color = 'orange' if any(x in vals[2] for x in vals[1]) else 'red'
    # print(f'pv_color: {pv_color}')
    colors = ['orange', 'red', 'red']
    symbols = ['x', 'x', 'circle-open']
    line_widths = [0.2, 0.2, 2.5]
    sizes = [7, 7, 10]
    d = dict(val=vals, color=colors, symbol=symbols, 
             lw=line_widths, size=sizes)
    pv_d = {k: {k: v[i] for k, v in d.items()} 
            for i, k in enumerate(keys)}
    # print('pv_d:')
    # pprint(pv_d)
   
    for k, v in pv_d.items():
        X = v['val']
        fig.add_scatter(x=df.Close[X].index,
                        y=df.Close[X],
                        name=k,
                        mode='markers',
                        marker=dict(symbol=v['symbol'], 
                                    line=dict(color=v['color'],
                                              width=v['lw']),
                                    color=v['color'],
                                    size=v['size']),
                        opacity=0.75,
                        )
    
   # show valid peaks & valleys     
    P, V = valid_peaks_valleys(df)
    chg = trend_changepoints_debug(df)
    X = sorted(P + V)

    fig.add_scatter(x=df.Close[X].index,
                    y=df.Close[X],
                    name='valid peaks/valleys',
                    mode='markers',
                    marker=dict(symbol='circle-open', 
                            color='darkgreen', 
                            size=12, 
                            line_width=2.5), 
                    opacity=0.75,
                    showlegend=False)
    
    # show trend changepoints
    for k, v in chg.items():
        try:
            x = max(v['begin'], v['end'])[-1]
            for x0, x1 in zip_longest(v['begin'], v['end'], fillvalue=x):
                if k == 'up':
                    txt = 'U'
                    color = 'green'
                elif k == 'down':
                    txt = 'D'
                    color = 'red'
                else:
                    txt = 'R'
                    color = 'violet'
                fig.add_vrect(x0=df.index[x0], 
                              x1=df.index[x1], 
                              line_width=0, 
                              fillcolor=color, 
                              opacity=0.2,
                              annotation_text=txt, 
                              annotation_position="top left")
        except:
            pass
            
    us_holidays = pd.to_datetime(list(holidays.US(range(start.year, end.year + 1)).keys()))
    rangebreaks = []
    rangeselector = []

    if period == 'M1':
        rangeselector = dict(buttons=[
                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                dict(count=2, label="2y", step="year", stepmode="backward"),
                                dict(count=3, label="3y", step="year", stepmode="backward"),
                                dict(step="all")
                                ])
    elif period == 'W1':
        rangeselector = dict(buttons=[
                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                dict(step="all")
                                ])  
    elif period == 'D1':
        rangebreaks = [dict(bounds=["sat", "mon"])]
    else:
        us_holidays += pd.offsets.Hour(9) + pd.offsets.Minute(30)
        rangebreaks = [dict(bounds=[16, 9.5], pattern="hour"), 
                       dict(bounds=["sat", "mon"])]

    if rangebreaks:
        us_holidays = pd.to_datetime(sorted(list(set(us_holidays) - set(df.index))))
        rangebreaks.append(dict(values=us_holidays))
        fig.update_xaxes(rangebreaks=rangebreaks)

    if rangeselector:
        fig.update_layout(xaxis1=dict(rangeselector=rangeselector))

    # pprint(fig.layout)

    cname = SPY_info_df.loc[ticker.upper(), 'Security']
    title = f'{cname} ({ticker.upper()}) - {period}'

    fig.update_layout(title=dict(text=title, xanchor='left'), paper_bgcolor=bgcolor, plot_bgcolor=bgcolor)
    fig.layout.xaxis.rangeslider.visible = False
    
    return fig

In [47]:
bg = ['black', 'gray', 'slategray', 'beige']
ticker = 'cah'
period = 'D1'
plot_trends(ticker, period, bgcolor=bg[2])

first: P, peaks: 22, valleys: 22

2. ['06/01/23', '09/01/23']
Prev. Trend: None
p0: 80.56, p1: 77.00, p2: 78.64               
v0: 76.73, v1: 76.62, v2: 77.57
Trend: RANGING
         Peaks Valleys    Trend
22/12/22 80.56       -  ranging
03/01/23     -   76.73  ranging
04/01/23 77.00       -  ranging
05/01/23     -   76.62  ranging
06/01/23 78.64       -  ranging
09/01/23     -   77.57  ranging
05/06/23 84.76       -  ranging
07/06/23     -   83.27  ranging

{'down': {'begin': [], 'end': []},
 'ranging': {'begin': [6], 'end': []},
 'up': {'begin': [], 'end': []}}
------------------------------
Start 22/12/22

Removed from ranging[begin]: (6 - 22/12/22)


3. ['10/01/23', '17/01/23']
Prev. Trend: RANGING
p0: 77.00, p1: 78.64, p2: 78.13               
v0: 76.62, v1: 77.57, v2: 76.42
Trend: RANGING
Continue RANGING
17/01/23 - 76.42
         Peaks Valleys    Trend
22/12/22 80.56       -  ranging
03/01/23     -   76.73  ranging
04/01/23 77.00       -  ranging
05/01/23     -   76.62  ranging


In [30]:
bonds_df = pd.read_csv(r'data\T-Bill Rates.csv', index_col=0).dropna()
crb_df = pd.read_csv(r'data\CRB Index.csv', index_col=0)
start = '2015-01-01'
end = bonds_df.index[-1]
bonds_df = bonds_df[start:end]
SPY_df = SPY_df[start:end]
crb_df.columns

Index(['Value'], dtype='object')

In [31]:
nrows = 3
row_heights = [1 / nrows] * nrows
fig = make_subplots(rows=nrows, cols=1,
                    shared_xaxes=True, 
                    vertical_spacing=0.05,
                    subplot_titles=[''] * nrows, 
                    row_heights=row_heights)
    
fig.add_scatter(x=bonds_df.index, 
                 y=bonds_df.Close, 
                 name='T-Bills', 
                 mode='lines',
                 line_color='blue', 
                #  opacity=0.5, 
                 connectgaps=True)
fig.add_scatter(x=SPY_df.index, 
                 y=SPY_df.Close, 
                 name='SPY', 
                 mode='lines',
                 line_color='green', 
                #  opacity=0.5, 
                 connectgaps=True,
                 row=2, col=1)
fig.add_scatter(x=crb_df.index, 
                 y=crb_df.Value, 
                 name='CRB', 
                 mode='lines',
                 line_color='violet', 
                #  opacity=0.5, 
                 connectgaps=True,
                 row=3, col=1)

fig.update_yaxes(showgrid=False, type='log')
fig.layout.yaxis1.autorange = "reversed"
# fig.layout.yaxis1.type = "log"
fig

### Peak Trough Progressions

 __Major Technical Principles__ 
- Never go for perfection; always shoot
for consistency.

- The significance of a peak-and-trough
reversal is determined by the duration and magnitude of the rallies and
reactions in question.

In [32]:
from sklearn import preprocessing
import time
pd.options.display.float_format = "{:,.2f}".format

In [95]:
def peak_valley_signals_debug(df, column='Close'):
    df = df.copy()
    prev_date = df.iloc[0].name
    unit = 'minutes' if prev_date.minute != 0 else 'days'
    X = df[column]
    P, V = valid_peaks_valleys(df, column=column)
    chg = trend_changepoints(df, column=column)
    nr = df.shape[0]
    t = None # trend
    first = 'P' if P[0] < V[0] else 'V'
    # lst, n = min((P, len(P)), (V, len(V)), key=itemgetter(1))
    rvsls = [] # list of ranges of trend reversal points
    df[['isPeak', 'isValley', 'Trend']] = 0
    df.loc[df.index[P], 'isPeak'] = 1
    df.loc[df.index[V], 'isValley'] = 1
    d_trend = {'up': 1, 'down': -1, 'ranging': 0}

    for k, v in chg.items():
        t = d_trend[k]
        for a, b in zip(v['begin'], v['end']):
            df.loc[df.index[a:b], 'Trend'] = t
    
    df['Trend Reversal'] = df['Trend'].diff()
    df['PV Reversal Signal'] = 0
    # t_rvsls = [i for i in range(nr) if df.loc[df.index[i], 'Trend Reversal'] != 0]
    # print(t_rvsls)

    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}')
    
    d = chg['ranging']
    scaled = {'Date': [], 'Duration': [], 'Size': []}

    # Identify trend reversal points
    for x in d['begin']:
        for k in ['up', 'down']:
            try:
                ix = chg[k]['end'].index(x)
                y = chg[k]['begin'][ix] # trend beginning
                rvsls.append((k, [x, y]))
                # print(f'Reversal from {k} - {x} - ({date_format(df.index[x])}, {X[x]:.2f}) ' \
                #       f'- {y} - ({date_format(df.index[y])}, {X[y]:.2f})')
                break
            except Exception as e:
                # print(e)
                pass
    
    for i, (k, v) in enumerate(rvsls):
        a, pa = v
        pa_date = df.index[pa]
        ix, iy = [bisect.bisect_left(x, a) for x in [P, V]]
        p_ix, p_iy = [x - 1 if x > 0 else 0 for x in [ix, iy]]
        pp, pv, p, v = X[[P[p_ix], V[p_iy], P[ix], V[iy]]]
        s_ix = f'({ix},{P[ix]})'
        s_iy = f'({iy},{V[iy]})'
        s_ix1 = f'({p_ix},{P[p_ix]})'
        s_iy1 = f'({p_iy},{V[p_iy]})'
        # if next trend is up, find next peak after chg_point & vice versa
        b = P[ix + 1] if k == 'down' else V[iy + 1]
        m = V[iy] if k == 'up' else P[ix]
        n = V[iy + 1] if k == 'down' else P[ix + 1]
        hs = 0.5
        s = 1
        half_sig = False
        print(f"{'-' * 50}"
              f'\n\n{k.upper()}, (a: {a}, b: {b})')
        print(f"P{get_sub(s_ix1)}: ({date_format(df.index[P[p_ix]])}, {X[P[p_ix]]:.2f}), "
              f"P{get_sub(s_ix)}: ({date_format(df.index[P[ix]])}, {X[P[ix]]:.2f})"
              f"\nV{get_sub(s_iy1)}: ({date_format(df.index[V[p_iy]])}, {X[V[p_iy]]:.2f}), "
              f"V{get_sub(s_iy)}: ({date_format(df.index[V[iy]])}, {X[V[iy]]:.2f})"
              f'\nm{get_sub(str(m))}: ({date_format(df.index[m])}, {X[m]:.2f})'
              f'\nb{get_sub(str(b))}: ({date_format(df.index[b])}, {X[b]:.2f})'
              f"\n{'-' * 50}")
        for j in range(a + 1, b + 1):
            date = df.index[j]
            s_date = df.index[b]
            print(f'X{get_sub(str(j))}: ({date_format(df.index[j])}, {X[j]:.2f})')
            if k == 'down':
                # MIGHT BE INCOMPLETE i.e. LACKING FULL REVERSAL SIGNAL
                if X[j] >= pp:
                    df.loc[date, 'PV Reversal Signal'] = hs
                    dur = date - pa_date
                    size = X[j] - X[pa]
                    scaled['Date'].append(df.index[j])
                    scaled['Duration'].append(dur)
                    scaled['Size'].append(abs(size))
                    print(f'Half Signal: X[j] >= pp \n(pv: {pv:.2f}, v: {v:.2f}), (pp: {pp:.2f}, p: {p:.2f})\n')
                    print(f'Duration: {dur.days} days, Size: {size:.2f}')
                    if X[n] >= p:
                        df.loc[df.index[n], 'PV Reversal Signal'] = s
                        print(f'Full signal: X[n] >= p \nX[n]: ({date_format(df.index[n])}, {X[n]:.2f})')
                    break
                if j > m and X[j] >= v:
                    if not half_sig:
                        df.loc[date, 'PV Reversal Signal'] = hs
                        print(f'Half Signal: j > m and X[j] >= v \n(pv: {pv:.2f}, v: {v:.2f}), (pp: {pp:.2f}, p: {p:.2f})\n')
                        t = df.loc[s_date, 'Trend']
                        dur = date - pa_date
                        size = X[j] - X[pa]
                        scaled['Date'].append(df.index[j])
                        scaled['Duration'].append(dur)
                        scaled['Size'].append(abs(size))
                        print(f'Duration: {dur.days} days, Size: {size:.2f}')
                        half_sig = True
                    if t != -s and X[j] >= p:
                        print('Reversed\n')
                        df.loc[date, 'PV Reversal Signal'] = s
                        dur = date - pa_date
                        size = X[j] - X[pa]
                        scaled['Date'].append(df.index[j])
                        scaled['Duration'].append(dur)
                        scaled['Size'].append(abs(size))
                        print(f'Duration: {dur.days} days, Size: {size:.2f}')
                        break
            
            if k == 'up':
                # MIGHT BE INCOMPLETE i.e. LACKING FULL REVERSAL SIGNAL
                if X[j] <= pv:
                    df.loc[date, 'PV Reversal Signal'] = -hs
                    dur = date - pa_date
                    size = X[j] - X[pa]
                    scaled['Date'].append(df.index[j])
                    scaled['Duration'].append(dur)
                    scaled['Size'].append(abs(size))
                    print(f'Half Signal: X[j] <= pv \n(pv: {pv:.2f}, v: {v:.2f}), (pp: {pp:.2f}, p: {p:.2f})\n')
                    print(f'Duration: {dur.days} days, Size: {size:.2f}')
                    if X[n] <= v:
                        df.loc[df.index[n], 'PV Reversal Signal'] = -s
                        print(f'Full signal: X[n] <= v \nX[n]: ({date_format(df.index[n])}, {X[n]:.2f})')
                    break
                if j > m and X[j] <= p:
                    if not half_sig:
                        df.loc[date, 'PV Reversal Signal'] = -hs
                        print(f'Half Signal: j > m and X[j] <= p \n(pv: {pv:.2f}, v: {v:.2f}), (pp: {pp:.2f}, p: {p:.2f})\n')
                        t = df.loc[s_date, 'Trend']
                        dur = date - pa_date
                        size = X[j] - X[pa]
                        scaled['Date'].append(df.index[j])
                        scaled['Duration'].append(dur)
                        scaled['Size'].append(abs(size))
                        print(f'Duration: {dur.days} days, Size: {size:.2f}')
                        half_sig = True
                    if t != s and X[j] <= v:
                        print('Reversed\n')
                        df.loc[date, 'PV Reversal Signal'] = -s
                        dur = date - pa_date
                        size = X[j] - X[pa]
                        scaled['Date'].append(df.index[j])
                        scaled['Duration'].append(dur)
                        scaled['Size'].append(abs(size))
                        print(f'Duration: {dur.days} days, Size: {size:.2f}')
                        break
    
    d = scaled['Duration']
    for i, dur in enumerate(d):
        d[i] = dur.days if unit == 'days' else dur.total_seconds() / 60
    
    col = 'Date'
    pv_df = pd.DataFrame(scaled, index=scaled[col]).drop(columns=col)
    scaler = preprocessing.MinMaxScaler(feature_range=(1, 10))
    sd = scaler.fit_transform(pv_df)
    scaled_df = pd.DataFrame(sd, columns=pv_df.columns, index=scaled[col])
    scaled_df['Signal'] = scaled_df.mean(axis=1)
    print(f'\n{scaled_df}\n')
    
    for i in scaled_df.index:
        df.loc[i, 'PV Signal Weight'] = scaled_df.loc[i, 'Signal']
        x = df.loc[i, 'PV Reversal Signal']
        y = scaled_df.loc[i, 'Signal']
        z = x * y 
        print(f"{date_format(i)}, {x:.2f}, {y:.2f}, {z:.2f}")
    
    return df[df['PV Reversal Signal'] != 0]

In [96]:
ticker = 'cah'
print(ticker.upper(), '\n')
period = 'D1'
days = TA_PERIODS[period]['days']
end = last_date
start = last_date - timedelta(days)
end = end + timedelta(1) if period.endswith('m') else end
df = make_dataframe(ticker, period=period)[start:end]
# sig_df = peak_valley_signals_debug(df, column='Close')
# sig_df.head(20)
# sig_df.tail()
pv_df = peak_valley_signals_debug(df)

CAH 

first: P, peaks: 22, valleys: 22
--------------------------------------------------

DOWN, (a: 26, b: 31)
P₍₅,₂₄₎: (20/01/23, 75.82), P₍₆,₂₈₎: (26/01/23, 77.88)
V₍₄,₂₃₎: (19/01/23, 75.48), V₍₅,₂₆₎: (24/01/23, 74.85)
m₂₈: (26/01/23, 77.88)
b₃₁: (31/01/23, 77.25)
--------------------------------------------------
X₂₇: (25/01/23, 75.73)
X₂₈: (26/01/23, 77.88)
Half Signal: X[j] >= pp 
(pv: 75.48, v: 74.85), (pp: 75.82, p: 77.88)

Duration: 16 days, Size: -0.25
--------------------------------------------------

UP, (a: 37, b: 59)
P₍₈,₃₄₎: (03/02/23, 78.67), P₍₉,₃₇₎: (08/02/23, 79.92)
V₍₈,₃₅₎: (06/02/23, 77.03), V₍₉,₃₈₎: (09/02/23, 78.26)
m₃₈: (09/02/23, 78.26)
b₅₉: (13/03/23, 70.10)
--------------------------------------------------
X₃₈: (09/02/23, 78.26)
X₃₉: (10/02/23, 79.83)
Half Signal: j > m and X[j] <= p 
(pv: 77.03, v: 78.26), (pp: 78.67, p: 79.92)

Duration: 9 days, Size: 4.11
X₄₀: (13/02/23, 79.46)
X₄₁: (14/02/23, 78.75)
X₄₂: (15/02/23, 78.58)
X₄₃: (16/02/23, 78.64)
X₄₄: (17

In [82]:
pv_df

,Open,High,Low,Close,Adj Close,Volume,isPeak,isValley,Trend,Trend Reversal,PV Reversal Signal,PV Signal Weight
Date,,,,,,,,,,,,
2022-05-06,144.51,154.66,143.78,149.53,140.80,15706400,1,0,-1,-1.00,0.50,1.00
2022-08-12,148.03,152.30,147.19,152.24,144.80,13374100,1,0,-1,-1.00,0.50,6.01
2022-11-25,126.64,129.82,126.23,129.04,125.45,7142700,1,0,0,0.00,-0.50,6.85
2022-12-09,125.07,129.44,123.31,125.76,122.26,12884300,0,0,0,0.00,-1.00,7.06


## S/R Levels

In [37]:
from sklearn import preprocessing
import time
# pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [38]:
end = last_date
years = 1
days = 180
start = end - timedelta(days * years)
ticker = 'aapl'
df = get_ticker_data(ticker)[start:end]

In [39]:
def fibonacci_retracement_levels(df):
    highest_swing = -1
    lowest_swing = -1
    high = df['High']
    low = df['Low']
    nr = df.shape[0]

    for i in range(1, nr - 1):
        if high[i] > high[i - 1] and high[i] > high[i + 1] \
            and (highest_swing == -1 or high[i] > high[highest_swing]):
            highest_swing = i
        if low[i] < low[i - 1] and low[i] < low[i + 1] \
            and (lowest_swing == -1 or low[i] < low[lowest_swing]):
            lowest_swing = i

    ratios = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
    levels = []
    max_level = high[highest_swing]
    min_level = low[lowest_swing]

    for ratio in ratios:
        # Uptrend
        if highest_swing > lowest_swing:
            level = max_level - (max_level - min_level) * ratio
        # Downtrend
        else:
            level = min_level + (max_level - min_level) * ratio
        levels.append(level)

    frl_data = {}
    unit = 'minutes' if df.iloc[0].name.minute != 0 else 'days'  
    prev_date = df.iloc[0].name

    for i in range(nr):
        date = df.iloc[i].name
        s_date = date.strftime('%d-%m-%y')
        l = bisect.bisect_left(levels, low[i])
        h = bisect.bisect_left(levels, high[i])
        h = h - 1 if h > 0 else h
        frls = [levels[h], levels[l]] if h != l else [levels[h]]
        # Deals with multiple levels being tested by one bar
        for frl in frls:
            if high[i] > frl and low[i] < frl:
                cum_vol = df.loc[prev_date:date, 'Volume'].sum()
                delta = date - prev_date # time it takes level to form
                delta = delta.days if unit == 'days' else delta.total_seconds() / 60
                prev_date = date
                d = {'Date': [], 'Timedelta': [], 'Volume': [], 'Tested': 0}
                frl_data.setdefault(frl, d)
                frl_data[frl]['Date'].append(s_date)
                frl_data[frl]['Timedelta'].append(delta)
                frl_data[frl]['Volume'].append(cum_vol)
                frl_data[frl]['Tested'] += 1             

    # Calculate significance of levels       
    d = {'FRL': [], 'Date': [], 'Timedelta': [], 'Volume': [], 'Tested': []}

    for k, v in frl_data.items():
        d['FRL'].append(k)
        d['Volume'].append(sum(v['Volume']))
        d['Timedelta'].append(sum(v['Timedelta']))
        d['Tested'].append(v['Tested'])
        d['Date'].append(v['Date'][-1])
        # print(v['Date'][-1])

    # NOT YET DECIDED HOW TO CALCULATE FRL SIGNAL

    return ratios, levels, frl_data


In [40]:
r, l, d = fibonacci_retracement_levels(df)
d

IndexError: list index out of range

In [7]:
def isSupport(df, i):
    '''Returns True if value is a price support level'''

    X = df['Low']
    support = X[i] < X[i - 1] \
                and X[i] < X[i + 1] \
                and X[i + 1] < X[i + 2] \
                and X[i - 1] < X[i - 2]

    return support


def isResistance(df, i):
    '''Returns True if value is a price resistance level'''

    X = df['High']
    resistance = X[i] > X[i - 1] \
                    and X[i] > X[i + 1] \
                    and X[i + 1] > X[i + 2] \
                    and X[i - 1] > X[i - 2] 

    return resistance
    

In [75]:
def sr_levels(df):
    '''Returns key support/resistance levels for a security'''

    df = df.copy()
    df['SR Signal'] = 0
    prev_date = df.iloc[0].name
    unit = 'minutes' if prev_date.minute != 0 else 'days'  
    spt, rst = (0, 0)
    levels = []
    s_levels = []
    sr_data = {}
    many_tests = {} # dict of bars that test more than 1 level
    s = (df['High'] - df['Low']).mean()
    nr, nc = df.shape

    def isFarFromLevel(l):
        '''Returns True if price is not near a previously discovered support or resistance'''
        
        return np.sum([abs(l - x[1]) < s for x in levels]) == 0

    for i in range(2, nr):
        date = df.iloc[i].name
        s_date = date.strftime('%d-%m-%y')
        high = df['High'][i]
        low = df['Low'][i]
        close = df['Close'][i]
        new_spt = False
        new_rst = False
        sr_switch = False

        if i < nr - 2:
            if isSupport(df, i):
                if isFarFromLevel(low):
                    new_spt = True
                    spt = low
                    df.loc[date, 'Support'] = spt
                    levels.append((i, spt))
                    s_levels = sorted([x[1] for x in levels])
                    # print('NS'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
            
            if isResistance(df, i):
                if isFarFromLevel(high):
                    new_rst = True
                    rst = high
                    df.loc[date, 'Resistance'] = rst
                    levels.append((i, rst))
                    s_levels = sorted([x[1] for x in levels])
                    # print('NR'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f},')    

        # Switch support to resistance & vice versa
        if len(levels) > 1:
            if new_spt:
                ix = bisect.bisect(s_levels, spt)
                rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
            if new_rst:
                ix = bisect.bisect_left(s_levels, rst)
                spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]
            if low > rst: # When resistance broken 
                sr_switch = True
                spt = rst
                ix = bisect.bisect(s_levels, low)
                rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
                # print('R-S'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
            if high < spt: # When support broken 
                sr_switch = True
                rst = spt
                ix = bisect.bisect_left(s_levels, high)
                spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]
                # print('S-R'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
        
        if new_rst or new_spt or sr_switch:
            cum_vol = df.loc[prev_date:date, 'Volume'].sum()
            delta = date - prev_date # time it takes level to form
            delta = delta.days if unit == 'days' else delta.total_seconds() / 60
            prev_date = date
            d = {'Date': [], 'Timedelta': [], 'Volume': [], 'SR': [], 'Tested': 0, 'Tested Date': []}
            sr_data.setdefault(spt, d)
            sr_data[spt]['Date'].append(s_date)
            sr_data[spt]['Timedelta'].append(delta)
            sr_data[spt]['Volume'].append(cum_vol)
            sr_data[spt]['SR'].append('S')
            # Prevents double-counting when support == resistance
            if spt != rst:
                sr_data.setdefault(rst, d)
                sr_data[rst]['Date'].append(s_date)
                sr_data[rst]['Timedelta'].append(delta)
                sr_data[rst]['Volume'].append(cum_vol)
                sr_data[rst]['SR'].append('R')
                                  
        if spt:
            if close < spt:
                df.loc[date, 'SR Signal'] = 1 # Generate signal
            # Check if S/R levels are tested       
            if high > spt and low < spt:
                sr_data[spt]['Tested'] += 1
                sr_data[spt]['Tested Date'].append(s_date)
                # print('ST'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                ix = bisect.bisect_left(s_levels, spt)
                n_spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]    
                while low < n_spt and spt != rst and spt != n_spt:
                    # print(f'SH-SL - {date.date()} - NS: {n_spt:.2f}, S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                    rst = spt
                    spt = n_spt
                    many_tests.setdefault(i, set()).union([spt, rst])
                    if ix > 0:
                        ix -= 1
                        n_spt = s_levels[ix]
                        sr_data[n_spt]['Date'].append(s_date)
                        sr_data[n_spt]['Timedelta'].append(delta)
                        sr_data[n_spt]['Volume'].append(cum_vol)
                        sr_data[n_spt]['SR'].append('S')            
   
        if rst:
            if close > rst:
                df.loc[date, 'SR Signal'] = 1 # Generate signal
            # Check if S/R levels are tested       
            if high > rst and low < rst:
                if spt != rst: # Prevents double-counting
                    sr_data[rst]['Tested'] += 1
                    sr_data[rst]['Tested Date'].append(s_date)
                    # print('RT'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                    ix = bisect.bisect(s_levels, rst)
                    n_rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
                    while high > n_rst and spt != rst and rst != n_rst:
                        # print(f'RL-RH - {date.date()} - NR: {n_rst:.2f}, R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                        spt = rst
                        rst = n_rst
                        many_tests.setdefault(i, set()).union([spt, rst])
                        if ix < len(s_levels) - 1:
                            ix += 1
                            # print(f'ix: {ix}, {s_levels}')
                            n_rst = s_levels[ix]
                            sr_data[n_rst]['Date'].append(s_date)
                            sr_data[n_rst]['Timedelta'].append(delta)
                            sr_data[n_rst]['Volume'].append(cum_vol)
                            sr_data[n_rst]['SR'].append('R')        

        if spt and rst: 
            df.loc[date:, 'Support'] = spt
            df.loc[date:, 'Resistance'] = rst

    del sr_data[0]     
    d = {'SR Level': [], 'Volume': [], 'Timedelta': [], 'Tested': [], 'Date': []}

    for k, v in sr_data.items():
        d['SR Level'].append(k)
        d['Volume'].append(sum(v['Volume']))
        d['Timedelta'].append(sum(v['Timedelta']))
        d['Tested'].append(v['Tested'])
        d['Date'].append(v['Date'][-1])

    ix = 'SR Level'
    sr_df = pd.DataFrame(d, index=d[ix]).drop(columns=ix)
    sr_df['Date'] = pd.to_datetime(sr_df['Date'])
    sr_df['Date'] = sr_df['Date'].apply(convert_to_timestamp)
    scaler = preprocessing.MinMaxScaler(feature_range=(1, 10))
    sd = scaler.fit_transform(sr_df)
    scaled_df = pd.DataFrame(sd, columns=sr_df.columns, index=d[ix])
    scaled_df['Signal'] = scaled_df.mean(axis=1)
    # print(sr_df)
    # print(scaled_df)
    
    # Make 'SR Signal' last column
    cols = list(df.columns)
    cols.append(cols.pop(cols.index('SR Signal')))
    df = df[cols]
    nr, nc = df.shape
    j = nc - 1 # 'SR Signal' column num

    # Assign significance to signals by level
    for i in range(nr):
        if df['SR Signal'][i]:
            if i in many_tests:
                signal = 0
                for l in many_tests[i]:
                    signal += scaled_df.loc[l, 'Signal']
            else:
                if df['Close'][i] > df['Resistance'][i]:
                    l = df['Resistance'][i]
                if df['Close'][i] < df['Support'][i]:
                    l = df['Support'][i]
                signal = scaled_df.loc[l, 'Signal']
 
            df.iloc[i, j] = signal        
    
    return levels, df, sr_data

In [77]:
pv_df

,Open,High,Low,Close,Adj Close,Volume,isPeak,isValley,Trend,Trend Reversal,PV Reversal Signal,PV Signal Weight
Date,,,,,,,,,,,,
2022-05-06,144.51,154.66,143.78,149.53,140.80,15706400,1,0,-1,-1.00,0.50,1.00
2022-05-27,144.58,149.52,141.00,149.51,142.21,11255800,1,0,-1,0.00,1.00,NaN
2022-08-12,148.03,152.30,147.19,152.24,144.80,13374100,1,0,-1,-1.00,0.50,6.01
2022-10-28,117.53,126.67,113.15,126.60,121.65,20565900,1,0,1,0.00,1.00,NaN
2022-11-25,126.64,129.82,126.23,129.04,125.45,7142700,1,0,0,0.00,-0.50,6.85
2022-12-09,125.07,129.44,123.31,125.76,122.26,12884300,0,0,0,0.00,-1.00,7.06


In [76]:
# sr_data.clear()
levels, ta_df, sr_data = sr_levels(df)
# levels
ta_df[ta_df['SR Signal'] != 0]

,Open,High,Low,Close,Adj Close,Volume,Resistance,Support,SR Signal
Date,,,,,,,,,
2021-12-03,176.47,176.83,169.18,172.59,160.96,17306600,203.21,173.78,6.43
2022-01-21,178.18,180.87,172.56,172.65,161.02,10675600,181.78,173.78,6.43
2022-01-28,171.86,175.25,160.54,163.10,152.11,23177200,181.78,173.78,6.43
2022-02-04,162.70,169.25,160.10,160.73,149.90,17924800,173.78,173.78,6.43
2022-02-11,160.75,164.26,158.80,159.54,148.79,14224500,173.78,173.78,6.43
2022-02-18,159.35,159.59,147.24,147.62,139.00,23666700,173.78,173.78,6.43
2022-02-25,148.20,151.76,139.74,150.51,141.72,21195500,173.78,173.78,6.43
2022-03-04,149.29,149.99,144.15,146.73,138.16,18303200,173.78,173.78,6.43
2022-03-11,145.74,149.47,140.83,140.96,132.73,20440000,173.78,173.78,6.43


In [ ]:
pprint(sr_data)

{134.3699951171875: {'Date': ['13-10-22',
                              '03-11-22',
                              '07-12-22',
                              '15-12-22',
                              '20-12-22',
                              '23-12-22',
                              '27-12-22',
                              '28-12-22',
                              '29-12-22',
                              '30-12-22',
                              '03-01-23',
                              '04-01-23',
                              '05-01-23',
                              '06-01-23',
                              '09-01-23',
                              '10-01-23',
                              '11-01-23',
                              '12-01-23',
                              '18-01-23'],
                     'SR': ['S',
                            'S',
                            'S',
                            'S',
                            'S',
                            'S',
   

In [66]:
d = {'SR Level': [], 'Volume': [], 'Timedelta': [], 'Tested': []}

for k, v in sr_data.items():
    d['SR Level'].append(k)
    d['Volume'].append(sum(v['Volume']))
    d['Timedelta'].append(sum(v['Timedelta']))
    d['Tested'].append(v['Tested'])

sr_df = pd.DataFrame(d, index=d['SR Level']).drop(columns='SR Level')
scaler = preprocessing.MinMaxScaler(feature_range=(1, 10))
sd = scaler.fit_transform(sr_df)
cols = ['Volume', 'Timedelta', 'Tested']
scaled_df = pd.DataFrame(sd, columns=sr_df.columns, index=d['SR Level'])
scaled_df['Signal'] = scaled_df.mean(axis=1)
print(sr_df.head(), '\n')
print(scaled_df)

NameError: name 'sr_data' is not defined

## MA Signals

- MA Crossovers, i.e. MA5/MA25, etc...
- Identify when moving averages converge

In [ ]:
def calc_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0] 
    
    return slope

In [ ]:
def calculate_ma_signals(df, MAs):
    for i, ma in enumerate(MAs):
        ix = df.iloc[ma].name
        df[f'MA{ma}'] = df['Close'].rolling(ma).mean()
        df[f'MA{ma} Slope'] = df[f'Close'].rolling(ma).apply(calc_slope)
        # df[f'MA{ma} Slope Diff'] = df[f'MA{ma} Slope'].diff()
        df.loc[ix:, f'MA{ma} Signal'] = np.where(df['Close'][ma:] > df[f'MA{ma}'][ma:], 1, 0)
        df[f'MA{ma} Crossover'] = df[f'MA{ma} Signal'].diff()
        df.drop(columns=f'MA{ma} Signal', inplace=True)

    df.loc[:, 'MA Std. Dev'] = df.loc[:, [f'MA{ma}' for ma in MAs]].std(axis=1)
    
    return df

In [ ]:
MAs = [10, 20, 50]
ma_df = calculate_ma_signals(ta_df, MAs)
# ma_df.loc['2022-09-12':'2022-11-12', ['MA10 Slope',	'MA10 Slope Diff', 'MA20 Slope', 'MA50 Slope']]
ma_df.loc['2022-09-12':'2022-11-12', ['MA10', 'MA20', 'MA50', 'MA Std. Dev']]

,MA10,MA20,MA50,MA Std. Dev
Date,,,,
2022-09-19,NaN,NaN,NaN,NaN
2022-09-20,NaN,NaN,NaN,NaN
2022-09-21,NaN,NaN,NaN,NaN
2022-09-22,NaN,NaN,NaN,NaN
2022-09-23,NaN,NaN,NaN,NaN
2022-09-26,NaN,NaN,NaN,NaN
2022-09-27,NaN,NaN,NaN,NaN
2022-09-28,NaN,NaN,NaN,NaN
2022-09-29,NaN,NaN,NaN,NaN


In [ ]:
math.atan2(ma_df['MA10'].values, ma_df.index.values)

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
ma_df.loc['2022-10-03':'2022-10-04', ['MA10', 'MA10 Slope', 'MA20', 'MA20 Slope']]

,MA10,MA10 Slope,MA20,MA20 Slope
2022-10-03,148.93,-0.73,152.09,NaN
2022-10-04,147.85,-0.83,151.67,NaN


In [ ]:
x = ma_df.loc['2022-10-03':'2022-10-04', 'MA10']
slope = np.polyfit(range(len(x)), x, 1)[0] 
slope


-1.0799987792968864

### Bollinger Bands

In [ ]:
from talib import BBANDS, MACD, RSI

In [ ]:
up, mid, down = BBANDS(df['Close'], timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)
bands = {'BB Up': up, 'BB Mid': mid, 'BB Down': down}
bb_df = pd.DataFrame(bands).dropna()
bb_df

,BB Up,BB Mid,BB Down
Date,,,
2023-01-13,139.84,131.55,123.27
2023-01-17,137.99,131.21,124.42
2023-01-18,137.76,131.15,124.53
2023-01-19,137.88,131.18,124.49
2023-01-20,138.71,131.45,124.18
...,...,...,...
2023-06-06,180.77,174.89,169.01
2023-06-07,181.06,175.09,169.12
2023-06-08,181.72,175.51,169.30


In [ ]:
from talib import BBANDS, MACD, RSI

In [ ]:
up, mid, low = BBANDS(df.Close, timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)

In [ ]:
# df[['BB Up', 'BB Mid', 'BB Down']] = up, mid, low
df['BB Up'] = up
df['BB Mid'] = mid
df['BB Down'] = low
df

,Open,High,Low,Close,Adj Close,Volume,BB Up,BB Mid,BB Down
Date,,,,,,,,,
2022-09-19,149.31,154.56,149.10,154.48,153.99,81474200,NaN,NaN,NaN
2022-09-20,153.40,158.08,153.08,156.90,156.40,107689800,NaN,NaN,NaN
2022-09-21,157.34,158.74,153.60,153.72,153.23,101696800,NaN,NaN,NaN
2022-09-22,152.38,154.47,150.91,152.74,152.25,86652500,NaN,NaN,NaN
2022-09-23,151.19,151.47,148.56,150.43,149.95,96029900,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2023-03-10,150.21,150.94,147.61,148.50,148.50,68524400,156.04,150.43,144.82
2023-03-13,147.81,153.14,147.70,150.47,150.47,84457100,156.02,150.41,144.80
2023-03-14,151.28,153.40,150.10,152.59,152.59,73695900,156.16,150.48,144.81


In [ ]:
rsi = RSI(df.Close, timeperiod=14)
rsi.index

DatetimeIndex(['2022-09-19', '2022-09-20', '2022-09-21', '2022-09-22',
               '2022-09-23', '2022-09-26', '2022-09-27', '2022-09-28',
               '2022-09-29', '2022-09-30',
               ...
               '2023-03-03', '2023-03-06', '2023-03-07', '2023-03-08',
               '2023-03-09', '2023-03-10', '2023-03-13', '2023-03-14',
               '2023-03-15', '2023-03-16'],
              dtype='datetime64[ns]', name='Date', length=124, freq=None)

In [ ]:
macd, macdsignal, macdhist = MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)

## Volume Analysis

In [ ]:
ticker = 'acn'
print('Ticker:', ticker.upper())
period = 'D1'
days = TA_PERIODS[period]['days']
end = last_date
start = last_date - timedelta(days)
end = end + timedelta(1) if period.endswith('m') else end
df = make_dataframe(ticker, period=period)[start:end]

Ticker: ACN


In [ ]:
def volume_analysis(df, ma=3):
    df = df.copy()
    df['% Chg Price'] = df['Close'].pct_change()
    df[f'Volume MA{ma}'] = df['Volume'].rolling(ma).mean()
    df[f'% Chg Volume MA{ma}'] = df[f'Volume MA{ma}'].pct_change()
    p = np.sign(df['% Chg Price'])
    v = np.sign(df[f'% Chg Volume MA{ma}'])
    df['Price-Volume'] = np.where(p == v, 0, 1)

    cols = ['Close', f'Volume MA{ma}', '% Chg Price', 
            f'% Chg Volume MA{ma}', 'Price-Volume']
    df = df[cols]
    nr, nc = df.shape
    nr1 = df[df['Price-Volume'] == 1].shape[0]
    print(f'{nr1 / nr:.0%} of Price-Volume == 1')

    df['Volume Signal'] = 0

    for i in range(nr):
        ix = df.index[i]
        if df.loc[ix, 'Price-Volume'] == 0:
            if p[i] > 0 and v[i] < 0:
                df.loc[i, 'Volume Signal'] = -1

    df.dropna(inplace=True)
    
    return df


In [ ]:
vol_df = volume_analysis(df, 3)
vol_df.head(30)

48% of Price-Volume == 1


,Close,Volume MA3,% Chg Price,% Chg Volume MA3,Price-Volume
Date,,,,,
2022-12-14,291.4500,NaN,NaN,NaN,1
2022-12-15,281.1200,NaN,-0.0354,NaN,1
2022-12-16,264.4800,"3,984,900.0000",-0.0592,NaN,1
2022-12-19,255.4000,"4,516,666.6667",-0.0343,0.1334,1
2022-12-20,261.9000,"4,594,166.6667",0.0255,0.0172,0
2022-12-21,268.9200,"3,610,033.3333",0.0268,-0.2142,1
2022-12-22,264.7600,"2,986,266.6667",-0.0155,-0.1728,0
2022-12-23,266.0900,"2,356,533.3333",0.0050,-0.2109,1
2022-12-27,265.3100,"1,499,700.0000",-0.0029,-0.3636,0


In [ ]:
# def volume_analysis(ticker, period, start, end):
#     MAs = TA_PERIODS[period]['MA']
#     short_ma, inter_ma, primary_ma, *_ = MAs
#     df = make_dataframe(ticker, period)[start:end]
#     nrows = df.shape[0]
#     df = df[['Close', 'Volume']]
#     df.rename(columns={'Close': 'Price'}, inplace=True)
#     df['Price %'] = df['Price'].pct_change()
#     df['Volume %'] = df['Volume'].pct_change()
#     p = np.sign(df['Price %'])
#     v = np.sign(df['Volume %'])
#     df['Price-Volume'] = np.where(p == v, 0, 1)
#     nrows1 = df[df['Price-Volume'] == 1].shape[0]
#     print(f'{nrows1 / nrows:.0%} of Price-Volume == 1')

#     for ma in MAs[:3]:
#         comp = []
#         for col in ('Price', 'Volume'):
#             s1 = f'{col} MA{ma}'
#             s2 = f'{s1} %'
#             df[s1] = df[col].rolling(ma).mean()
#             df[s2] = df[s1].pct_change()
#             X = np.sign(df[s2])
#             comp.append(X)
#         p, v = comp
#         s = f'Price-Volume MA{ma}'
#         df[s] = np.where(p == v, 0, 1)
#         nrows1 = df[df[s] == 1].shape[0]
#         print(f'{nrows1 / nrows:.0%} of {s} == 1')

#     # df.dropna(inplace=True)

#     # cols = ['Price %', 'Volume %', 
#     #         # 'Close Trend', 'Volume Trend',
#     #         'Price-Volume']
#     # df = df[cols]

#     return df

In [ ]:
vol_df = volume_analysis(ticker, period, start, end)
vol_df.tail(20)

55% of Price-Volume == 1
62% of Price-Volume MA10 == 1
72% of Price-Volume MA20 == 1
78% of Price-Volume MA40 == 1


,Price,Volume,Price %,Volume %,Price-Volume,Price MA10,Price MA10 %,Volume MA10,Volume MA10 %,Price-Volume MA10,Price MA20,Price MA20 %,Volume MA20,Volume MA20 %,Price-Volume MA20,Price MA40,Price MA40 %,Volume MA40,Volume MA40 %,Price-Volume MA40
Date,,,,,,,,,,,,,,,,,,,,
2022-11-04,136.0800,6768100,-0.0194,0.2383,1,129.8430,0.0048,"6,874,150.0000",0.0161,0,128.7030,0.0092,"6,949,300.0000",-0.0150,1,128.7743,-0.0002,"8,458,147.5000",-0.0087,0
2022-11-11,148.3100,6032300,0.0899,-0.1087,1,131.8730,0.0156,"6,720,680.0000",-0.0223,1,130.0540,0.0105,"6,763,140.0000",-0.0268,1,128.9540,0.0014,"8,438,047.5000",-0.0024,1
2022-11-18,146.1900,6744600,-0.0143,0.1181,1,132.7290,0.0065,"6,738,300.0000",0.0026,0,131.4030,0.0104,"6,756,860.0000",-0.0009,1,129.1788,0.0017,"8,442,612.5000",0.0005,0
2022-11-25,156.9600,7291200,0.0737,0.0810,0,135.4360,0.0204,"6,591,200.0000",-0.0218,1,133.1255,0.0131,"6,840,635.0000",0.0124,0,129.8015,0.0048,"8,276,070.0000",-0.0197,1
2022-12-02,155.6700,7764200,-0.0082,0.0649,1,138.6550,0.0238,"6,737,570.0000",0.0222,0,134.9675,0.0138,"6,919,050.0000",0.0115,0,130.3500,0.0042,"8,130,970.0000",-0.0175,1
2022-12-09,152.9500,5618200,-0.0175,-0.2764,0,141.7950,0.0226,"6,592,210.0000",-0.0216,1,136.3630,0.0103,"6,872,275.0000",-0.0068,1,130.8265,0.0037,"7,895,570.0000",-0.0290,1
2022-12-16,149.3000,7870600,-0.0239,0.4009,1,143.9810,0.0154,"6,686,090.0000",0.0142,0,137.1230,0.0056,"6,876,995.0000",0.0007,0,131.3013,0.0036,"7,834,677.5000",-0.0077,1
2022-12-23,149.2300,6019800,-0.0005,-0.2352,0,146.3340,0.0163,"6,580,900.0000",-0.0157,1,137.9040,0.0057,"6,869,740.0000",-0.0011,1,131.5540,0.0019,"7,760,327.5000",-0.0095,1
2022-12-30,149.6500,3218000,0.0028,-0.4654,1,148.3110,0.0135,"6,279,280.0000",-0.0458,1,138.7090,0.0058,"6,770,535.0000",-0.0144,1,131.9025,0.0026,"7,657,012.5000",-0.0133,1


In [ ]:
def plot_signals(graph, ticker, start, end, period, plot_data,
                 show_vol, show_rsi, show_macd, show_sr, show_fr, 
                 show_bb, show_MAs, show_adv_MAs, show_trend_analysis, 
                 show_trendlines_c, show_trendlines_hl):
    '''
    Returns plot figure

    Parameters
    ----------
    df: DataFrame of a security's market data
    start: Start Date
    end: End Date
    period: user-input
    minor_ma: moving average window
    secondary_ma : moving average window
    primary_ma: moving average window
    show_prices: display candlestick data on crowded charts
    '''

    ticker = ticker.split(' - ')[0]
    end = end + timedelta(1) if period.endswith('m') else end
    df = make_dataframe(ticker, period)[start:end]
    MAs = plot_data['MAs']
    nrows = 1 + show_vol + show_rsi + show_macd
    r1 = 1 - 0.1 * nrows
    r2 = (1 - r1) / (nrows - 1)
    row_heights = [r1] + [r2] * (nrows - 1)
    fig_row = 2
    data = []
    name = ticker

    fig = make_subplots(rows=nrows, cols=1,
                        shared_xaxes=True, 
                        vertical_spacing=0.05,
                        subplot_titles=[''] * nrows, 
                        row_heights=row_heights)
    fig.update_xaxes(showgrid=True)          
    fig.update_yaxes(showgrid=False, type='log')

    if graph == 'Candlesticks' and not show_trend_analysis and not show_trendlines_c:
        cs = go.Candlestick(x=df.index, 
                            open=df['Open'], 
                            high=df['High'],
                            low=df['Low'], 
                            close=df['Close'],
                            name=name)  
        cs.increasing.fillcolor = 'green'
        cs.increasing.line.color = 'darkgreen'
        cs.decreasing.fillcolor = 'red'
        cs.decreasing.line.color = 'indianred'
        data.append(cs)
    else:
        xb = go.Scatter(x=df.index,
                        y=df['Close'],
                        name=name,
                        line_width=1.5,
                        connectgaps=True)
        data.append(xb)

    # Plot MAs & advanced MAs
    if show_MAs or show_adv_MAs:
        adv_MAs = plot_data['Adv MAs']
        colors = ['red', 'cyan', 'gold']
        dash = 'dot' if show_MAs and show_adv_MAs else 'solid'
        for ma, adv_ma, color in zip(MAs, adv_MAs, colors): 
            if show_MAs:
                y = df['Close'].rolling(ma).mean()
                sma = go.Scatter(x=df.index,
                                 y=y,
                                 name=f'MA{ma}',
                                 line_width=1,
                                 line_color=color,
                                 connectgaps=True)
                data.append(sma)
            if show_adv_MAs:
                y = df['Close'].rolling(ma).mean().shift(adv_ma)
                advanced_sma = go.Scatter(x=df.index,
                                          y=y,
                                          name=f'MA{ma}+{adv_ma}',
                                          line_width=1.25,
                                          line_color=color,
                                          line_dash=dash,
                                          connectgaps=True)
                data.append(advanced_sma)
    
    pos = [1] * len(data) # position to add rows, cols in subplot 
    fig.add_traces(data=data, rows=pos, cols=pos)  

    # Support & resistance lines
    if show_sr:
        levels, _ = sr_levels(df)
        for i, l in levels:
            n = df.shape[0] - i
            fig.add_scatter(x=df.index[i:],
                            y=[l] * n,
                            name='SR',
                            line_width=0.5,
                            line_color='orange',
                            mode='lines',
                            showlegend=False,
                            connectgaps=True)   

    # Fibonacci retracements
    if show_fr:
        colors = ["darkgray", "indianred", "green", "blue", "cyan", "magenta", "gold"]
        ratios, levels = fibonacci_retracements(df)
        for i in range(len(ratios)):
            fig.add_scatter(x=df.index,
                            y=[levels[i]] * df.shape[0],
                            name=f'FR {ratios[i]:,.2%}',
                            line_color=colors[i],
                            line_width=0.75,
                            line_dash='dot',
                            connectgaps=True)
    
    # Bollinger bands
    if show_bb:
        up, mid, down = BBANDS(df['Close'], timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)
        bands = {'BB Up': up, 'BB Mid': mid, 'BB Down': down}
        for k, v in bands.items():
            dash = 'dot' if k == 'BB Mid' else 'solid'
            fig.add_scatter(x=v.index,
                            y=v.values,
                            name=k,
                            line_width=1,
                            line_dash=dash,
                            mode='lines',
                            connectgaps=True)

    # Trend Analysis
    if show_trend_analysis:
        fig.update_xaxes(showgrid=False)
        fig.update_yaxes(showgrid=False)
        # P, V, r_PV = order_peaks_valleys(df)
        vals = order_peaks_valleys(df)
        keys = ['peaks', 'valleys', 'removed']
        colors = ['orange', 'red', 'red']
        symbols = ['x', 'x', 'circle-x']
        line_widths = [0.2, 0.2, 2.5]
        sizes = [7, 7, 10]
        d = dict(val=vals, color=colors, symbol=symbols, 
                 lw=line_widths, size=sizes)
        pv_d = {k: {k: v[i] for k, v in d.items()} 
                for i, k in enumerate(keys)}

        # show all peaks/valleys & removed peaks/valleys    
        for k, v in pv_d.items():
            X = v['val']
            fig.add_scatter(x=df.Close[X].index,
                            y=df.Close[X],
                            name=k,
                            mode='markers',
                            marker=dict(symbol=v['symbol'], 
                                        line=dict(color=v['color'],
                                                  width=v['lw']),
                                        color=v['color'],
                                        size=v['size']),
                            opacity=0.75,
                            )

        # show valid peaks & valleys        
        chg, peaks, valleys = trend_changepoints(df)
        X = sorted(peaks + valleys)

        fig.add_scatter(x=df.Close[X].index,
                        y=df.Close[X],
                        name='valid peaks/valleys',
                        mode='markers',
                        marker=dict(symbol='circle-open', 
                                color='limegreen', 
                                size=12, 
                                line_width=2.5), 
                        opacity=0.5,
                        showlegend=False)
        
        # show trend changepoints
        for k, v in chg.items():
            try:
                x = max(v['start'], v['end'])[-1]
                for x0, x1 in zip_longest(v['start'], v['end'], fillvalue=x):
                    if k == 'up':
                        txt = 'U'
                        color = 'green'
                    elif k == 'down':
                        txt = 'D'
                        color = 'red'
                    else:
                        txt = 'R'
                        color = 'gray'
                    fig.add_vrect(x0=df.index[x0], 
                                  x1=df.index[x1], 
                                  line_width=0, 
                                  fillcolor=color, 
                                  opacity=0.2,
                                  annotation_text=txt, 
                                  annotation_position="top left")
            except: pass      

    # Trendlines
    if show_trendlines_c or show_trendlines_hl:
        pv_df, peaks, valleys, PV, trendlines_c, trendlines_hl = peaks_valleys_trendlines(df)
        fig.add_scatter(x=df.Close[peaks].index,
                        y=df.Close[peaks],
                        name='Peaks',
                        mode='markers',
                        marker=dict(symbol='x', color='yellow', size=5))
        fig.add_scatter(x=df.Close[valleys].index,
                        y=df.Close[valleys],
                        name='Valleys',
                        mode='markers',
                        marker=dict(symbol='x', color='red', size=5))
        fig.add_scatter(x=df.Close[PV].index,
                        y=df.Close[PV],
                        name='Valid Peaks / Valleys',
                        mode='markers',
                        marker=dict(symbol='circle-open', color='limegreen', size=8))
        if show_trendlines_c:      
            for x, y in trendlines_c:
                fig.add_scatter(x=x,
                                y=y,
                                name='Trendline (Close)',
                                mode='lines',
                                line_color='magenta',
                                opacity=0.5,
                                showlegend=False)
        if show_trendlines_hl:      
            for x, y in trendlines_hl:
                fig.add_scatter(x=x,
                                y=y,
                                name='Trendline (HL)',
                                mode='lines',
                                line_color='cyan',
                                opacity=0.5,
                                showlegend=False)

    # Volume subplot
    if show_vol:
        name = 'Volume'
        fig.add_bar(x=df.index,
                    y=df[name],
                    name=name,
                    marker={'color': 'steelblue'},
                    row=fig_row, col=1)
        # fig.update_layout({f'yaxis{fig_row}': {'title': name}})
        fig_row += 1

    # RSI subplot
    if show_rsi:
        rsi = RSI(df['Close'], timeperiod=14)
        name = 'RSI'
        fig.add_scatter(x=rsi.index,
                        y=rsi.values,
                        name=name,
                        line_width=1,
                        mode='lines',
                        connectgaps=True,
                        showlegend=False,
                        row=fig_row, col=1)
        fig.add_hline(70, line_width=0.5, line_dash='dot', line_color='red', 
                      row=fig_row, col=1)
        fig.add_hline(30, line_width=0.5, line_dash='dot', line_color='red', 
                      row=fig_row, col=1)
        fig.update_layout({f'yaxis{fig_row}': {'type': 'linear', 'title': name}})
        fig_row += 1
    
    # MACD subplot
    if show_macd:
        macd, *_ = MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        name = 'MACD'
        fig.add_scatter(x=macd.index,
                        y=macd.values,
                        name=name,
                        line_width=1,
                        mode='lines',
                        connectgaps=True,
                        showlegend=False,
                        row=fig_row, col=1)
        fig.update_layout({f'yaxis{fig_row}': {'type': 'linear', 'title': name}})
    
    us_holidays = pd.to_datetime(list(holidays.US(range(start.year, end.year + 1)).keys()))
    rangebreaks = []
    rangeselector = []

    if period == 'M1':
        rangeselector = dict(buttons=[
                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                dict(count=2, label="2y", step="year", stepmode="backward"),
                                dict(count=3, label="3y", step="year", stepmode="backward"),
                                dict(step="all")
                                ])
    elif period == 'W1':
        rangeselector = dict(buttons=[
                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                dict(step="all")
                                ])  
    elif period == 'D1':
        rangebreaks = [dict(bounds=["sat", "mon"])]
    else:
        us_holidays += pd.offsets.Hour(9) + pd.offsets.Minute(30)
        rangebreaks = [dict(bounds=[16, 9.5], pattern="hour"), 
                       dict(bounds=["sat", "mon"])]

    if rangebreaks:
        us_holidays = pd.to_datetime(sorted(list(set(us_holidays) - set(df.index))))
        rangebreaks.append(dict(values=us_holidays))
        fig.update_xaxes(rangebreaks=rangebreaks)

    if rangeselector:
        fig.update_layout(xaxis1=dict(rangeselector=rangeselector))

    cname = SPY_info_df.loc[ticker, 'Security']
    title = f'{cname} ({ticker}) - {period}'

    fig.update_layout(title=dict(text=title, xanchor='center'))
    fig.layout.xaxis.rangeslider.visible = False

    return fig

In [ ]:
ticker = 'A'
period = 'D1'
df = make_dataframe(ticker, period)
graph = 'line'
plot_MAs = TA_PERIODS[period]['MA']
plot_data = {'MAs': plot_MAs,
             'Adv MAs': [int(ma**(1/2)) for ma in plot_MAs]}
end = df.index[-1]
days = TA_PERIODS[period]['days']
start = end - timedelta(days=days)
show_vol = True 
show_rsi = False
show_macd = False
show_sr = False
show_fr = False
show_bb = False
show_MAs = False
show_adv_MAs = False
show_trend_analysis = False 
show_trendlines_c = False
show_trendlines_hl = False

In [ ]:
plot_signals(graph, ticker, start, end, period, plot_data,
             show_vol, show_rsi, show_macd, show_sr, show_fr, 
             show_bb, show_MAs, show_adv_MAs, show_trend_analysis, 
             show_trendlines_c, show_trendlines_hl)